- In this Notebook, I am trying to integrate the training data from the backscore and rest of the period from prj-prod-dataplatform.audit_balance.ml_model_run_details table.
- In this I will compare the training period with each month of test period.

**Steps to Follow**:

* Read the specific model data from prj-prod-dataplatform.audit_balance.ml_model_run_details table
* Expand the calcFeature column to extract all the features for the model
* Read the data from specific backscore table for the training data
* Identify the features and create a list
* Use transform_data function to create the same structure as ml_model_run_details table
* Insert the data to a similar training table - prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
* Read the specific model data from prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
* expand the training set from the calcFeature column
* Concatenate both the test and train datasets
* Calculate the PSI using the PSI function comparing it with the train set
* Insert the result to a PSI table prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi_v4

# **PSI - CSI Calculation**

## Define Libraries

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

## Function

In [2]:
# Method 1: Using regex to remove all "Calc_" occurrences
def clean_names_regex(name):
    return re.sub(r'_Calc_', '_', name)

# Method 2: Simple string replacement
def clean_names_replace(name):
    return name.replace('_Calc_', '_')

## expand_calc_features_robust

In [3]:
import pandas as pd
import json

def expand_calc_features_robust(df):
    """
    Expand the calcFeatures JSON column into separate columns and return the complete DataFrame.
    
    Parameters:
    df (pd.DataFrame): Input DataFrame with calcFeatures column containing JSON data
    
    Returns:
    pd.DataFrame: Expanded DataFrame with all original columns plus JSON features as separate columns
    """
    
    # Make a copy to avoid modifying the original DataFrame
    df_expanded = df.copy()
    
    # Parse the calcFeatures JSON column
    calc_features_data = []
    
    for idx, calc_features_str in enumerate(df['calcFeatures']):
        try:
            # Handle None or NaN values
            if pd.isna(calc_features_str):
                calc_features_data.append({})
                continue
            
            # Convert to string if not already
            calc_features_str = str(calc_features_str)
            
            # Parse the JSON string
            features_dict = json.loads(calc_features_str.replace("'", '"'))
            
            # Ensure it's a dictionary, not a list
            if isinstance(features_dict, dict):
                calc_features_data.append(features_dict)
            elif isinstance(features_dict, list):
                # If it's a list, convert to dict with index keys or skip
                print(f"Warning: calcFeatures at index {idx} is a list, converting to dict")
                calc_features_data.append({'raw_list': features_dict})
            else:
                print(f"Warning: calcFeatures at index {idx} is neither dict nor list: {type(features_dict)}")
                calc_features_data.append({})
                
        except (json.JSONDecodeError, AttributeError, TypeError) as e:
            # If parsing fails, create an empty dict and print warning
            print(f"Warning: Could not parse calcFeatures at index {idx}: {e}")
            print(f"  Value: {calc_features_str}")
            calc_features_data.append({})
    
    # Create DataFrame from the parsed JSON data
    calc_features_df = pd.DataFrame(calc_features_data)
    
    # Add prefix to JSON-derived columns to avoid conflicts
    calc_features_df = calc_features_df.add_prefix('calc_')
    
    # Reset index to ensure proper alignment
    df_expanded = df_expanded.reset_index(drop=True)
    calc_features_df = calc_features_df.reset_index(drop=True)
    
    # Combine original DataFrame with expanded calcFeatures
    result_df = pd.concat([df_expanded, calc_features_df], axis=1)
    
    return result_df

### dropping_duplicates

In [4]:
def dropping_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drop duplicates based on digitalLoanAccountId, Data_selection, and modelVersionid,
    keeping the first occurrence based on appln_submit_datetime.

    Parameters:
    -----------
    df : pd.DataFrame
        Input dataframe

    Returns:
    --------
    pd.DataFrame with duplicates dropped
    """

    df = df.sort_values(
        ['digitalLoanAccountId', 'Data_selection', 'modelVersionId', 'appln_submit_datetime'],
        ascending=[True, True, True, True],
        na_position='last'
    )

    result = df.drop_duplicates(
        subset=['digitalLoanAccountId', 'Data_selection', 'modelVersionId'],
        keep='first'
    ).copy()

    return result

### PSI Pipeline Version 1

In [5]:
# import pandas as pd
# import numpy as np
# from datetime import datetime
# from typing import List, Dict, Optional, Tuple

# class PSICalculator:
#     """
#     A class to calculate Population Stability Index (PSI) with support for 
#     multiple segments and score columns.
#     """
    
#     def __init__(self, df: pd.DataFrame):
#         """
#         Initialize the PSI Calculator with a combined dataframe.
        
#         Parameters:
#         -----------
#         df : pd.DataFrame
#             Combined dataframe with train and test data
#         """
#         self.df = df.copy()
#         self._prepare_data()
    
#     def _prepare_data(self):
#         """Prepare and clean the data for PSI calculation."""
#         # Convert score columns to numeric
#         if 'score' in self.df.columns:
#             self.df['score'] = pd.to_numeric(self.df['score'], errors='coerce')
        
#         # Convert Application_month to datetime
#         if 'Application_month' in self.df.columns:
#             if self.df['Application_month'].dtype != 'datetime64[ns]':
#                 self.df['Application_month'] = pd.to_datetime(
#                     self.df['Application_month'] + '-01'
#                 )
        
#         # Standardize Data_selection column (handle case sensitivity)
#         if 'Data_selection' in self.df.columns:
#             self.df['Data_selection'] = self.df['Data_selection'].str.strip().str.title()
    
#     def _calculate_decile_bins(self, train_scores: pd.Series) -> np.ndarray:
#         """
#         Calculate decile bins from training data.
        
#         Parameters:
#         -----------
#         train_scores : pd.Series
#             Training set scores
        
#         Returns:
#         --------
#         np.ndarray
#             Array of bin edges for deciles
#         """
#         _, bins = pd.qcut(train_scores.dropna(), 10, labels=False, retbins=True)
#         return bins
    
#     def _get_distribution(
#         self, 
#         scores: pd.Series, 
#         bins: np.ndarray
#     ) -> pd.Series:
#         """
#         Get the decile distribution for a set of scores.
        
#         Parameters:
#         -----------
#         scores : pd.Series
#             Scores to bin
#         bins : np.ndarray
#             Decile bin edges
        
#         Returns:
#         --------
#         pd.Series
#             Distribution across deciles (0-9)
#         """
#         deciles = pd.cut(scores.dropna(), bins=bins, labels=False, include_lowest=True)
#         distribution = pd.Series(deciles).value_counts().sort_index() / len(scores)
        
#         # Align with all 10 deciles
#         all_bins = range(10)
#         distribution_aligned = distribution.reindex(all_bins, fill_value=0)
#         return distribution_aligned
    
#     def _calculate_psi_value(
#         self, 
#         current_dist: pd.Series, 
#         baseline_dist: pd.Series
#     ) -> float:
#         """
#         Calculate PSI value given two distributions.
        
#         Parameters:
#         -----------
#         current_dist : pd.Series
#             Current period distribution
#         baseline_dist : pd.Series
#             Baseline period distribution
        
#         Returns:
#         --------
#         float
#             PSI value
#         """
#         # Add small constant to avoid log(0)
#         epsilon = 1e-10
#         current_dist = current_dist + epsilon
#         baseline_dist = baseline_dist + epsilon
        
#         psi = (current_dist - baseline_dist) * np.log(current_dist / baseline_dist)
#         return psi.sum()
    
#     def calculate_psi_by_segment(
#         self,
#         score_column: str = 'score',
#         segment_columns: Optional[List[str]] = None,
#         date_column: str = 'Application_month',
#         data_selection_column: str = 'Data_selection',
#         account_id_column: str = 'digitalLoanAccountId'
#     ) -> pd.DataFrame:
#         """
#         Calculate PSI for each segment and month.
        
#         Parameters:
#         -----------
#         score_column : str
#             Name of the score column to analyze
#         segment_columns : Optional[List[str]]
#             List of columns to segment by. If None, calculates overall PSI only.
#             Example: ['modelVersionId', 'trenchCategory', 'loan_product_type']
#         date_column : str
#             Name of the application month column
#         data_selection_column : str
#             Name of the column distinguishing train/test data
#         account_id_column : str
#             Name of the account ID column for counting distinct accounts
        
#         Returns:
#         --------
#         pd.DataFrame
#             DataFrame with columns: Month, scorename, DateCategory, psivalues, 
#             account_count, segment_column, segment_value
#         """
        
#         # Validate inputs
#         if score_column not in self.df.columns:
#             raise ValueError(f"Score column '{score_column}' not found in dataframe")
        
#         if segment_columns is None:
#             segment_columns = []
        
#         # Check all required columns exist
#         required_cols = [date_column, data_selection_column, account_id_column, score_column]
#         missing_cols = [col for col in required_cols + segment_columns if col not in self.df.columns]
#         if missing_cols:
#             raise ValueError(f"Missing columns: {missing_cols}")
        
#         # Separate train and test data
#         train_df = self.df[self.df[data_selection_column].str.lower() == 'train'].copy()
#         test_df = self.df[self.df[data_selection_column].str.lower() == 'test'].copy()
        
#         if train_df.empty:
#             raise ValueError("No training data found")
#         if test_df.empty:
#             raise ValueError("No test data found")
        
#         # Calculate baseline decile bins from training data
#         train_bins = self._calculate_decile_bins(train_df[score_column])
#         baseline_dist = self._get_distribution(train_df[score_column], train_bins)
        
#         results = []
        
#         # Get last training month
#         last_train_month = train_df[date_column].max()
#         last_train_month_str = last_train_month.strftime('%Y-%m')
        
#         # Case 1: Overall PSI (no segments)
#         if not segment_columns:
#             # Add training set PSI
#             train_psi = self._calculate_psi_value(baseline_dist, baseline_dist)
#             results.append({
#                 'Month': last_train_month_str,
#                 'scorename': score_column,
#                 'DateCategory': 'Training',
#                 'psivalues': train_psi,
#                 'account_count': train_df[account_id_column].nunique(),
#                 'segment_column': None,
#                 'segment_value': None
#             })
            
#             # Calculate monthly PSI for test set
#             for month in sorted(test_df[date_column].unique()):
#                 month_str = month.strftime('%Y-%m')
#                 month_df = test_df[test_df[date_column] == month]
#                 if not month_df.empty:
#                     month_dist = self._get_distribution(month_df[score_column], train_bins)
#                     month_psi = self._calculate_psi_value(month_dist, baseline_dist)
#                     results.append({
#                         'Month': month_str,
#                         'scorename': score_column,
#                         'DateCategory': 'Monthly',
#                         'psivalues': month_psi,
#                         'account_count': month_df[account_id_column].nunique(),
#                         'segment_column': None,
#                         'segment_value': None
#                     })
        
#         # Case 2: Segmented PSI
#         else:
#             # For each segment column
#             for seg_col in segment_columns:
#                 # Training set PSI by segment
#                 for seg_val in train_df[seg_col].unique():
#                     if pd.isna(seg_val):
#                         continue
                    
#                     seg_train_df = train_df[train_df[seg_col] == seg_val]
#                     if not seg_train_df.empty:
#                         seg_baseline_dist = self._get_distribution(
#                             seg_train_df[score_column], 
#                             train_bins
#                         )
#                         seg_train_psi = self._calculate_psi_value(
#                             seg_baseline_dist, 
#                             seg_baseline_dist
#                         )
#                         results.append({
#                             'Month': last_train_month_str,
#                             'scorename': score_column,
#                             'DateCategory': 'Training',
#                             'psivalues': seg_train_psi,
#                             'account_count': seg_train_df[account_id_column].nunique(),
#                             'segment_column': seg_col,
#                             'segment_value': str(seg_val)
#                         })
                
#                 # Monthly PSI by segment for test set
#                 for month in sorted(test_df[date_column].unique()):
#                     month_str = month.strftime('%Y-%m')
#                     month_df = test_df[test_df[date_column] == month]
                    
#                     for seg_val in month_df[seg_col].unique():
#                         if pd.isna(seg_val):
#                             continue
                        
#                         seg_month_df = month_df[month_df[seg_col] == seg_val]
#                         if not seg_month_df.empty:
#                             seg_month_dist = self._get_distribution(
#                                 seg_month_df[score_column], 
#                                 train_bins
#                             )
#                             seg_month_psi = self._calculate_psi_value(
#                                 seg_month_dist, 
#                                 baseline_dist
#                             )
#                             results.append({
#                                 'Month': month_str,
#                                 'scorename': score_column,
#                                 'DateCategory': 'Monthly',
#                                 'psivalues': seg_month_psi,
#                                 'account_count': seg_month_df[account_id_column].nunique(),
#                                 'segment_column': seg_col,
#                                 'segment_value': str(seg_val)
#                             })
        
#         return pd.DataFrame(results)


# # Usage Examples:
# # ===============

# # Example 1: Overall PSI (no segments)
# # psi_calc = PSICalculator(df_

### PSI Pipeline Version 2

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
from itertools import product, combinations

def calculate_psi_with_segments(
    df,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=None,
    num_bins=10,
    train_label='Train',
    test_label='Test'
):
    """
    Calculate Population Stability Index (PSI) with waterfall segmentation support.
    Creates waterfall combinations: first segment alone, then segment1+segment2, 
    then segment1+segment2+segment3, etc.
    Each segment uses its own training distribution.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Combined dataframe with train and test data
    score_column : str
        Name of the score column (default: 'score')
    date_column : str
        Name of the date column (default: 'Application_month')
    data_selection_column : str
        Name of the column indicating train/test split (default: 'Data_selection')
    segment_columns : list
        List of column names to segment by in waterfall order.
        Example: ['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
    num_bins : int
        Number of bins/deciles for PSI calculation (default: 10)
    train_label : str
        Value indicating training data (default: 'Train')
    test_label : str
        Value indicating test data (default: 'Test')
    
    Returns:
    --------
    pd.DataFrame
        Output dataframe with columns: Month, scorename, DateCategory, psivalues, 
        account_count, modelVersionId, trenchCategory, new_loan_type, 
        loan_product_type, osType
    """
    
    # Create a copy to avoid modifying original data
    df = df.copy()
    
    # Convert score to numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    
    # Convert Application_month to datetime if needed
    if df[date_column].dtype != 'datetime64[ns]':
        df[date_column] = pd.to_datetime(df[date_column].astype(str) + '-01', errors='coerce')
    
    # Separate train and test data
    train_df = df[df[data_selection_column] == train_label].copy()
    test_df = df[df[data_selection_column] == test_label].copy()
    
    # Handle missing values
    train_df = train_df.dropna(subset=[score_column])
    test_df = test_df.dropna(subset=[score_column])
    
    if train_df.empty or test_df.empty:
        raise ValueError("Train or Test dataframe is empty after removing NaN values")
    
    # If no segment columns specified, calculate overall PSI
    if segment_columns is None or len(segment_columns) == 0:
        segment_columns = []
    
    # Function to calculate PSI with predefined bins
    def calculate_psi_with_bins(data_scores, train_distribution, decile_bins):
        """
        Calculate PSI using pre-defined decile bins from training data.
        """
        if len(data_scores) == 0:
            return np.nan
        
        # Bin the scores using the training bins
        data_binned = pd.cut(data_scores, bins=decile_bins, labels=False, include_lowest=True)
        
        # Get distribution of current data
        current_dist = pd.Series(data_binned).value_counts().sort_index() / len(data_scores)
        
        # Align with training distribution (ensure all bins are represented)
        all_bins = range(num_bins)
        current_dist_aligned = current_dist.reindex(all_bins, fill_value=1e-10)
        train_dist_aligned = train_distribution.reindex(all_bins, fill_value=1e-10)
        
        # Calculate PSI
        psi_values = (current_dist_aligned - train_dist_aligned) * np.log(current_dist_aligned / train_dist_aligned)
        
        return psi_values.sum()
    
    # Get the last month of the training set
    last_train_month = train_df[date_column].max()
    last_train_month_str = last_train_month.strftime('%Y-%m')
    
    # Initialize results list
    psi_results = []
    
    # Define all possible segment columns (for creating NA columns)
    all_possible_segments = ['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
    
    # If no segments, calculate overall PSI
    if len(segment_columns) == 0:
        # Calculate training deciles and distribution
        train_score_binned = pd.qcut(train_df[score_column], num_bins, labels=False, duplicates='drop')
        train_distribution = train_score_binned.value_counts().sort_index() / len(train_df[score_column])
        _, train_decile_bins = pd.qcut(train_df[score_column], num_bins, retbins=True, duplicates='drop')
        
        # Training set PSI
        train_psi = calculate_psi_with_bins(train_df[score_column], train_distribution, train_decile_bins)
        result_row = {
            'Month': last_train_month_str,
            'scorename': score_column,
            'DateCategory': 'Training',
            'psivalues': train_psi,
            'account_count': train_df['digitalLoanAccountId'].nunique(),
        }
        # Add all segment columns as NA
        for seg_col in all_possible_segments:
            result_row[seg_col] = 'NA'
        psi_results.append(result_row)
        
        # Monthly PSI for test set
        for month in sorted(test_df[date_column].unique()):
            month_str = month.strftime('%Y-%m')
            month_df = test_df[test_df[date_column] == month]
            
            if not month_df.empty:
                month_psi = calculate_psi_with_bins(month_df[score_column], train_distribution, train_decile_bins)
                result_row = {
                    'Month': month_str,
                    'scorename': score_column,
                    'DateCategory': 'Monthly',
                    'psivalues': month_psi,
                    'account_count': month_df['digitalLoanAccountId'].nunique(),
                }
                for seg_col in all_possible_segments:
                    result_row[seg_col] = 'NA'
                psi_results.append(result_row)
    
    else:
        # Create waterfall combinations
        # For waterfall: [col1], [col1, col2], [col1, col2, col3], etc.
        waterfall_combinations = []
        for i in range(1, len(segment_columns) + 1):
            waterfall_combinations.append(segment_columns[:i])
        
        # Process each waterfall level
        for current_segments in waterfall_combinations:
            
            # Get unique values for each segment column in current waterfall level
            segment_combinations = []
            for seg_col in current_segments:
                unique_vals = df[seg_col].dropna().unique()
                segment_combinations.append([(seg_col, val) for val in unique_vals])
            
            # Generate all combinations for this waterfall level
            for seg_combo in product(*segment_combinations):
                # Filter data for this segment combination
                seg_filter = df.copy()
                
                for seg_col, seg_val in seg_combo:
                    seg_filter = seg_filter[seg_filter[seg_col] == seg_val]
                
                if seg_filter.empty:
                    continue
                
                # Split by train/test for this segment
                seg_train = seg_filter[seg_filter[data_selection_column] == train_label].copy()
                seg_test = seg_filter[seg_filter[data_selection_column] == test_label].copy()
                
                # Skip if segment has no data
                if seg_train.empty or seg_test.empty:
                    continue
                
                # Remove NaN values
                seg_train = seg_train.dropna(subset=[score_column])
                seg_test = seg_test.dropna(subset=[score_column])
                
                if seg_train.empty or seg_test.empty:
                    continue
                
                # Calculate training distribution SPECIFIC to this segment
                seg_train_binned = pd.qcut(
                    seg_train[score_column],
                    num_bins,
                    labels=False,
                    duplicates='drop'
                )
                seg_train_dist = seg_train_binned.value_counts().sort_index() / len(seg_train[score_column])
                
                # Get decile bins specific to this segment's training data
                _, seg_train_decile_bins = pd.qcut(
                    seg_train[score_column],
                    num_bins,
                    retbins=True,
                    duplicates='drop'
                )
                
                # Training set PSI for this segment
                seg_train_psi = calculate_psi_with_bins(seg_train[score_column], seg_train_dist, seg_train_decile_bins)
                
                # Create result row for training
                result_row = {
                    'Month': last_train_month_str,
                    'scorename': score_column,
                    'DateCategory': 'Training',
                    'psivalues': seg_train_psi,
                    'account_count': seg_train['digitalLoanAccountId'].nunique(),
                }
                
                # Add segment values
                for seg_col, seg_val in seg_combo:
                    result_row[seg_col] = seg_val
                
                # Add NA for segments not in this waterfall level
                for seg_col in all_possible_segments:
                    if seg_col not in [col for col, _ in seg_combo]:
                        result_row[seg_col] = 'NA'
                
                psi_results.append(result_row)
                
                # Monthly PSI for test set (this segment)
                for month in sorted(seg_test[date_column].unique()):
                    month_str = month.strftime('%Y-%m')
                    month_df = seg_test[seg_test[date_column] == month]
                    
                    if not month_df.empty:
                        month_psi = calculate_psi_with_bins(month_df[score_column], seg_train_dist, seg_train_decile_bins)
                        
                        result_row = {
                            'Month': month_str,
                            'scorename': score_column,
                            'DateCategory': 'Monthly',
                            'psivalues': month_psi,
                            'account_count': month_df['digitalLoanAccountId'].nunique(),
                        }
                        
                        # Add segment values
                        for seg_col, seg_val in seg_combo:
                            result_row[seg_col] = seg_val
                        
                        # Add NA for segments not in this waterfall level
                        for seg_col in all_possible_segments:
                            if seg_col not in [col for col, _ in seg_combo]:
                                result_row[seg_col] = 'NA'
                        
                        psi_results.append(result_row)
    
    # Create output dataframe
    output_df = pd.DataFrame(psi_results)
    
    # Reorder columns
    column_order = ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count'] + all_possible_segments
    output_df = output_df[[col for col in column_order if col in output_df.columns]]
    
    return output_df


# USAGE EXAMPLES:
# ===============

# Example 1: Waterfall segmentation with all levels
# score_output_df = calculate_psi_with_segments(
#     df=df_combined,
#     score_column='score',
#     date_column='Application_month',
#     data_selection_column='Data_selection',
#     segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
# )

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

# print(f"Output shape: {score_output_df.shape}")
# print(f"\nOutput columns: {score_output_df.columns.tolist()}")
# print(f"\nFirst few rows:")
# print(score_output_df.head(20))
# print(f"\nData types:")
# print(score_output_df.dtypes)

# Config query

In [7]:
sq = """ WITH base AS (
  -- First part with actual trenchCategory
  SELECT 
    modelDisplayName, 
    modelVersionId, 
    CASE 
      WHEN trenchCategory IS NULL THEN 'ALL'
      WHEN trenchCategory = '' THEN 'ALL'
      ELSE trenchCategory 
    END AS trenchCategory,
(CASE 
      WHEN modelDisplayName LIKE '%sil%' THEN 'SIL'
      WHEN modelDisplayName LIKE '%cash%' THEN 'CASH'
      ELSE 'ALL' 
    END) AS product_category,
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`

  
  UNION ALL
  
  -- Second part with 'ALL' trenchCategory
  SELECT 
    modelDisplayName, 
    modelVersionId, 
    'ALL' AS trenchCategory,
    (CASE 
      WHEN modelDisplayName LIKE '%sil%' THEN 'SIL'
      WHEN modelDisplayName LIKE '%cash%' THEN 'CASH'
      ELSE 'ALL' 
    END) AS product_category,
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE trenchCategory IS NOT NULL

)
SELECT distinct
  modelDisplayName, 
  modelVersionId, 
  trenchCategory, 
  product_category,
FROM base
ORDER BY 4, 1, 2, 3;"""

configdf = client.query(sq).to_dataframe()
configdf

,modelDisplayName,modelVersionId,trenchCategory,product_category
0,alpha_income_model,v1,ALL,ALL
1,beta_income_model,v1,ALL,ALL
2,beta_income_model,v1,Trench 1,ALL
3,beta_income_model,v1,Trench 2,ALL
4,beta_income_model,v1,Trench 3,ALL
5,alpha_stack_model_cash,v1,ALL,CASH
6,alpha_stack_model_cash,v1,Trench 1,CASH
7,alpha_stack_model_cash,v1,Trench 2,CASH
8,alpha_stack_model_cash,v1,Trench 3,CASH
9,alpha_stack_model_cash,v1.1,ALL,CASH


In [8]:
# configdf.to_csv('configdf.csv', index = False)

## SIL

### Queries

### cic_model_sil

### Test

In [9]:
## This is for the test period of Alpha - CIC sil model - reading the data from ml_model_run_details

sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
        case when modelDisplayName = 'Alpha - CIC-SIL-Model' then 'cic_model_sil' else modelDisplayName end as modelDisplayName    
    ,modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  ),
base as
(SELECT distinct

--Alpha_cic_sil_score
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'cic_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
)
select * from base
;
"""
dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

## this data is not expanded. We will have to expand and get the features from the calcFeatures column

The shape of the dataframe is: (145056, 19)


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,2400379,eabfc953-9fa3-40a4-ae3d-8c3e14f41edb,0.11524232185137818,2025-11-28 11:28:46.101516,2025-11-28 11:28:46.108083,cic_model_sil,v1,SIL-Instore,M,Appliance,android,cic_model_sil,SIL,ALL,"{""run_date"":1764288000000,""cic_Personal_Loans_...",Test,2025-11-28 19:28:32,2025-11-28 19:43:36,2025-11
1,2540390,b5194cdb-8d38-4ef7-807b-b771d9be03e1,0.042163754735859205,2025-11-03 10:20:44.562903,2025-11-03 10:20:44.569501,cic_model_sil,v1,SIL ZERO,F,Appliance,android,cic_model_sil,SIL,ALL,"{""run_date"":1762128000000,""cic_Personal_Loans_...",Test,2025-11-03 18:20:33,2025-11-03 18:45:59,2025-11
2,2689160,2527683e-1eef-4103-8f9a-b68f48167df4,0.05890744965217573,2025-10-02 08:13:00.781415,2025-10-02 08:13:00.787929,cic_model_sil,v1,SIL-Instore,M,Appliance,android,cic_model_sil,SIL,ALL,"{""run_date"":1759363200000,""cic_Personal_Loans_...",Test,2025-10-02 16:12:51,NaT,2025-10
3,2842707,38565ac0-279a-42de-84a0-1fee64cf0c50,0.07453270364599196,2025-10-01 06:11:17.434365,2025-10-01 06:11:17.440731,cic_model_sil,v1,SIL-Instore,M,Mall,android,cic_model_sil,SIL,ALL,"{""run_date"":1759276800000,""cic_Personal_Loans_...",Test,2025-10-01 14:11:06,2025-10-04 17:34:33,2025-10
4,3155145,612fbd62-2807-47f6-b0ac-410c81dc06b2,0.06651499894571895,2025-10-28 09:47:06.928941,2025-10-28 09:47:06.935592,cic_model_sil,v1,SIL-Instore,F,Appliance,android,cic_model_sil,SIL,ALL,"{""run_date"":1761609600000,""cic_Personal_Loans_...",Test,2025-10-28 17:46:51,2025-10-28 17:49:45,2025-10


In [10]:
df1 = dfd.copy()

### Train

In [11]:
sq = """WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    
    case when modelDisplayName = 'Alpha - CIC-SIL-Model' then 'cic_model_sil' else modelDisplayName end as modelDisplayName
    
    ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - CIC-SIL-Model', 'cic_model_sil')
  ),
base as 
(SELECT distinct
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'Alpha - CIC-SIL-Model' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Train' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base
;
"""

dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

The shape of the dataframe is: (430892, 19)


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,1878137,e635c643-dd36-48cd-ad9e-d31b6417b722,0.125483,2025-12-23T22:16:31.552682,2025-12-23T22:16:31.552682,cic_model_sil,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,ALL,"{""cic_days_since_last_inquiry"": 1347.0, ""cic_z...",Train,2023-01-29 15:27:37,2023-01-29 15:38:12,2023-01
1,2027808,2fae7ea0-a02f-47cf-a30e-dd082ff5d95d,0.086636,2025-12-23T22:16:21.712002,2025-12-23T22:16:21.712002,cic_model_sil,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,ALL,"{""cic_Personal_Loans_granted_contracts_amt_24M...",Train,2025-01-22 15:55:42,2025-01-22 16:00:56,2025-01
2,2073563,49c25789-0664-46db-911d-1e9e295b4253,0.084308,2025-12-23T22:16:20.613333,2025-12-23T22:16:20.613333,cic_model_sil,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,ALL,"{""cic_Personal_Loans_granted_contracts_amt_24M...",Train,2023-08-20 17:14:53,2023-08-20 17:26:22,2023-08
3,2123425,2880f5d7-df7b-4bbc-9e68-34583c85907d,0.161857,2025-12-23T22:16:31.948099,2025-12-23T22:16:31.948099,cic_model_sil,v1,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,ALL,"{""cic_days_since_last_inquiry"": 199.0, ""cic_ve...",Train,2023-07-07 12:53:01,NaT,2023-07
4,2125254,f95c7cc3-8ca7-477b-991b-c170546aa96b,0.071070,2025-12-23T22:16:33.109456,2025-12-23T22:16:33.109456,cic_model_sil,v1,SIL-Instore,M,Appliance,android,Alpha - CIC-SIL-Model,SIL,ALL,"{""cic_Personal_Loans_granted_contracts_amt_24M...",Train,2024-10-08 12:00:52,NaT,2024-10


In [12]:
df2 = dfd.copy()

In [13]:
df2[['Application_month']][(df2['modelVersionId'] =='v1')].max()

Application_month    2025-01
dtype: object

In [14]:
df2[df2['Application_month'] == '2025-11']

,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
205130,3206286,6b2d3c9c-efd4-4d62-a4d3-db28826ffd7a,0.394028,2025-12-23T22:28:52.295109,2025-12-23T22:28:52.295109,cic_model_sil,v2,SIL-Instore,F,Appliance,android,Alpha - CIC-SIL-Model,SIL,Trench 1,"{""ScoreRange"": ""Missing"", ""ln_loan_level_user_...",Train,2025-11-15 10:23:43,2025-01-21 19:13:10,2025-11


In [15]:
dd.query("""select Application_month, count(distinct digitalLoanAccountId)cnt from df2 
         where modelVersionId ='v1'
         group by 1 order by 1 desc;""").to_df()

,Application_month,cnt
0,2025-01,16318
1,2024-12,34696
2,2024-11,17931
3,2024-10,16817
4,2024-09,17502
5,2024-08,17808
6,2024-07,13346
7,2024-06,13472
8,2024-05,8033
9,2024-04,5851


In [16]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (575948, 19)


In [17]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (575948, 19)
The shape of the dataframe after dropping duplicates is: (575948, 19)


In [18]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [19]:
df_combined['Application_month'][df_combined['Data_selection'] == 'Train'].max()

'2025-11'

### PSI calculation

In [20]:
# USAGE EXAMPLES:
# ===============

# Example 1: Calculate overall PSI (no segmentation)
# score_output_df = calculate_psi_with_segments(
#     df=df_combined,
#     score_column='score',
#     date_column='Application_month',
#     data_selection_column='Data_selection',
#     segment_columns=None
# )

# Example 2: Calculate PSI with single segmentation (modelVersionId)
# score_output_df = calculate_psi_with_segments(
#     df=df_combined,
#     score_column='score',
#     date_column='Application_month',
#     data_selection_column='Data_selection',
#     segment_columns=['modelVersionId']
# )

# Example 3: Calculate PSI with multiple segmentations (all combinations)
# score_output_df = calculate_psi_with_segments(
#     df=df_combined,
#     score_column='score',
#     date_column='Application_month',
#     data_selection_column='Data_selection',
#     segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
# )

# Example 4: With a different score column name
# score_output_df = calculate_psi_with_segments(
#     df=df_combined,
#     score_column='risk_score',  # Different score column
#     date_column='Application_month',
#     data_selection_column='Data_selection',
#     segment_columns=['modelVersionId', 'trenchCategory']
# )

# Example 1: Waterfall segmentation with all levels
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (482, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-11     score     Training   0.000000         246408             v2   
1   2025-11     score      Monthly   0.082671            944             v2   
2   2025-12     score      Monthly   0.064708          21069             v2   
3   2026-01     score      Monthly   0.058383          17419             v2   
4   2026-02     score      Monthly   0.065841           2193             v2   
5   2025-11     score     Training   0.000000         184484             v1   
6   2025-03     score      Monthly   0.134249            180             v1   
7   2025-04     score      Monthly   0.110920            541             v1   
8   2025-05     score      Monthly   0.108170            858             v1   

In [21]:
# # Initialize calculator
# psi_calc = PSICalculator(df_combined)
# results_overall = psi_calc.calculate_psi_by_segment(score_column='score')

In [22]:
# # 2. PSI by individual segment
# results_modelversion = psi_calc.calculate_psi_by_segment(
#     score_column='score',
#     segment_columns=['modelVersionId']
# )

In [23]:
# results_modelversion

In [24]:
# results_multi = psi_calc.calculate_psi_by_segment(
#     score_column='score',
#     segment_columns=['modelVersionId', 'trenchCategory', 'loan_product_type', 'osType']
# )

In [25]:
score_output_df

score_output_df['Model_Name'] = 'cic_model_sil'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'Alpha_cic_sil_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-11,Alpha_cic_sil_score,Training,0.000000,246408,v2,NA,NA,NA,NA,cic_model_sil
1,2025-11,Alpha_cic_sil_score,Monthly,0.082671,944,v2,NA,NA,NA,NA,cic_model_sil
2,2025-12,Alpha_cic_sil_score,Monthly,0.064708,21069,v2,NA,NA,NA,NA,cic_model_sil
3,2026-01,Alpha_cic_sil_score,Monthly,0.058383,17419,v2,NA,NA,NA,NA,cic_model_sil
4,2026-02,Alpha_cic_sil_score,Monthly,0.065841,2193,v2,NA,NA,NA,NA,cic_model_sil
...,...,...,...,...,...,...,...,...,...,...,...
477,2025-08,Alpha_cic_sil_score,Monthly,0.162986,90,v1,ALL,SIL ZERO,Appliance,ios,cic_model_sil
478,2025-09,Alpha_cic_sil_score,Monthly,0.422861,78,v1,ALL,SIL ZERO,Appliance,ios,cic_model_sil
479,2025-10,Alpha_cic_sil_score,Monthly,0.136165,69,v1,ALL,SIL ZERO,Appliance,ios,cic_model_sil
480,2025-11,Alpha_cic_sil_score,Monthly,0.255677,83,v1,ALL,SIL ZERO,Appliance,ios,cic_model_sil


In [26]:
# # 4. With custom column names
# results_custom = psi_calc.calculate_psi_by_segment(
#     score_column='score',
#     segment_columns=['modelVersionId', 'new_loan_type'],
#     account_id_column='digitalLoanAccountId',
#     data_selection_column='Data_selection'
# )

In [27]:
# copy the data to psi_results
psi_results = score_output_df.copy()

# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cic_sil_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=653843e6-c173-4479-8330-8dd397ceb736>

### Alpha Sil Stack Model

### Test

In [28]:
## This is for the test period of Alpha - CIC sil model - reading the data from ml_model_run_details
# Sil_Alpha_Stack_score
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
        case when modelDisplayName = 'Alpha - StackingModel' then 'alpha_stack_model_sil' else modelDisplayName end as modelDisplayName    
    ,modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
base as
(SELECT distinct
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'alpha_stack_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Test' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
)
select * from base
;
"""
dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

## this data is not expanded. We will have to expand and get the features from the calcFeatures column

The shape of the dataframe is: (145059, 19)


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,2306217,358dc89f-f60a-4ca8-8e13-136cf8abbb9b,0.1760408025300193,2025-11-16 01:23:33.445315,2025-11-16 01:23:33.451093,alpha_stack_model_sil,v1,SIL-Instore,M,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.1176522856, ""s_cic_score"":...",Test,2025-11-16 09:23:22,2025-11-16 09:25:47,2025-11
1,2503320,3de254d7-0b46-421a-a6a3-c8a53fd8c81a,0.1769802514313911,2025-07-18 06:55:10.776874,2025-07-18 06:55:10.783011,alpha_stack_model_sil,v1,SIL Competitor,M,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.1035514076, ""s_cic_score"":...",Test,2025-07-18 14:55:01,NaT,2025-07
2,2546841,d6cccdbd-5540-4974-8a2a-ca67f3cbf5a4,0.1232809699503863,2025-11-11 05:15:32.801479,2025-11-11 05:15:32.807655,alpha_stack_model_sil,v1,SIL ZERO,F,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.1845373542, ""s_cic_score"":...",Test,2025-11-11 13:15:21,NaT,2025-11
3,2820403,c8bb45c1-655a-4e35-8595-fcec249fb715,0.23615407829924287,2025-05-28 06:42:25.979081,2025-05-28 06:42:25.984002,alpha_stack_model_sil,v1,SIL-Instore,M,Mall,ios,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.1489104414, ""s_cic_score"":...",Test,2025-05-28 14:42:17,NaT,2025-05
4,3002473,763f9f74-7cf0-4a6a-acb6-5a6396684a44,0.10527837876709768,2025-08-01 11:22:38.453994,2025-08-01 11:22:38.459605,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.0971039402, ""s_cic_score"":...",Test,2025-08-01 19:22:27,2025-08-01 19:33:30,2025-08


In [29]:
df1 = dfd.copy()

### Train

In [30]:
sq = """WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    
    case when modelDisplayName = 'Alpha - StackingModel' then 'alpha_stack_model_sil' else modelDisplayName end as modelDisplayName 
     ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Alpha - StackingModel', 'alpha_stack_model_sil')
  ),
base as 
(SELECT distinct
  r.customerId,r.digitalLoanAccountId,prediction score
    ,start_time,end_time,modelDisplayName,modelVersionId,
   loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
        when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
        when lower(loanmaster.deviceType) like '%andro%' then 'android'
        else 'ios' end osType,
 'alpha_stack_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  r.calcFeature calcFeatures,
  'Train' Data_selection,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month,
FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base
;
"""

dfd = client.query(sq).to_dataframe()
print(f"The shape of the dataframe is: {dfd.shape}")
dfd.head()

The shape of the dataframe is: (439057, 19)


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,calcFeatures,Data_selection,appln_submit_datetime,disbursementDateTime,Application_month
0,1241521,eab64c50-007e-4049-b9ba-1fbd58764c79,0.187442,2025-12-23T22:17:00.301204,2025-12-23T22:17:00.301204,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,ios,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.06024873149825297, ""s_cic_...",Train,2024-05-04 14:31:22,2024-05-04 14:34:58,2024-05
1,1298347,2be00ec1-d10e-4f0f-8928-eab0e59d9349,0.131448,2025-12-23T22:16:59.435353,2025-12-23T22:16:59.435353,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,ios,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.3139038417705712, ""s_cic_s...",Train,2023-08-07 18:44:05,2023-08-07 18:50:12,2023-08
2,1644735,1c9d0925-a2c6-4270-9723-54a42ec05390,0.075888,2025-12-23T22:16:54.365461,2025-12-23T22:16:54.365461,alpha_stack_model_sil,v1,SIL-Instore,F,Mobile,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.18930936182401167, ""s_cic_...",Train,2024-11-28 15:07:36,NaT,2024-11
3,2039345,390cfaa8-123c-4979-a1b0-09681dd81d29,0.025545,2025-12-23T22:16:56.008202,2025-12-23T22:16:56.008202,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.08238934167115015, ""s_cic_...",Train,2023-05-11 14:19:16,2023-05-11 14:26:53,2023-05
4,2052062,7d2197fc-2a49-4c01-b7ef-e39f914a2a78,0.228661,2025-12-23T22:16:57.775467,2025-12-23T22:16:57.775467,alpha_stack_model_sil,v1,SIL-Instore,F,Appliance,android,alpha_stack_model_sil,SIL,ALL,"{""sb_demo_score"": 0.08599108652681178, ""s_cic_...",Train,2023-05-19 18:05:22,2023-05-19 18:14:32,2023-05


In [31]:
df2 = dfd.copy()

In [32]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (584116, 19)


In [33]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (584116, 19)
The shape of the dataframe after dropping duplicates is: (584116, 19)


In [34]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

### PSI calculation

In [35]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (482, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-11     score     Training   0.000000         254573             v2   
1   2025-11     score      Monthly   0.565769            944             v2   
2   2025-12     score      Monthly   0.043884          21069             v2   
3   2026-01     score      Monthly   0.040086          17419             v2   
4   2026-02     score      Monthly   0.048517           2196             v2   
5   2025-11     score     Training   0.000000         184484             v1   
6   2025-03     score      Monthly  12.685199            180             v1   
7   2025-04     score      Monthly   0.176158            541             v1   
8   2025-05     score      Monthly   0.041444            858             v1   

In [36]:
score_output_df

score_output_df['Model_Name'] = 'alpha_stack_model_sil'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'Sil_Alpha_Stack_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-11,Sil_Alpha_Stack_score,Training,0.000000,254573,v2,NA,NA,NA,NA,alpha_stack_model_sil
1,2025-11,Sil_Alpha_Stack_score,Monthly,0.565769,944,v2,NA,NA,NA,NA,alpha_stack_model_sil
2,2025-12,Sil_Alpha_Stack_score,Monthly,0.043884,21069,v2,NA,NA,NA,NA,alpha_stack_model_sil
3,2026-01,Sil_Alpha_Stack_score,Monthly,0.040086,17419,v2,NA,NA,NA,NA,alpha_stack_model_sil
4,2026-02,Sil_Alpha_Stack_score,Monthly,0.048517,2196,v2,NA,NA,NA,NA,alpha_stack_model_sil
...,...,...,...,...,...,...,...,...,...,...,...
477,2025-08,Sil_Alpha_Stack_score,Monthly,0.130898,90,v1,ALL,SIL ZERO,Appliance,ios,alpha_stack_model_sil
478,2025-09,Sil_Alpha_Stack_score,Monthly,0.212421,78,v1,ALL,SIL ZERO,Appliance,ios,alpha_stack_model_sil
479,2025-10,Sil_Alpha_Stack_score,Monthly,0.274605,69,v1,ALL,SIL ZERO,Appliance,ios,alpha_stack_model_sil
480,2025-11,Sil_Alpha_Stack_score,Monthly,0.326472,83,v1,ALL,SIL ZERO,Appliance,ios,alpha_stack_model_sil


In [37]:
psi_results = score_output_df.copy()
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_sil_stack_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=138122eb-ac16-4d47-86f8-bcd5dad4e051>

### Beta Sil App Score

### Test

In [38]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - AppsScoreModel' then 'apps_score_model_sil' else modelDisplayName end as modelDisplayName ,   
    modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
    
  ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'apps_score_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Test' Data_selection,
  -- sil_beta_app_score
  safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64) as score,
 calcFeature calcFeatures,
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid,  modelVersionId  order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
  )
select * from base 
;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1819064,ce042d34-1182-48c1-943c-c97348796765,"{'cb_score': 0.29711937823959866, 'dl_score': ...",2025-09-29 09:20:48.909020,2025-09-29 09:20:49.418653,apps_score_model_sil,v1,SIL-Instore,M,Appliance,android,apps_score_model_sil,SIL,ALL,Test,0.502240,"{""app_cnt_rated_for_3plus_ever"":43,""app_cnt_ed...",2025-09-29 17:20:44,NaT,2025-09
1,1886080,02a996d5-9ec2-4375-9cfc-7c75ba6ba5ac,0.42568453586745975,2025-12-24 10:03:50.679958,2025-12-24 10:03:51.343129,apps_score_model_sil,v2,SIL Competitor,M,Appliance,android,apps_score_model_sil,SIL,Trench 2,Test,NaN,"{""appScoreModel"": {""app_cnt_art_and_design_eve...",2025-12-24 18:03:46,NaT,2025-12
2,1921562,2f1cb945-2b56-48d0-828f-95352487308b,"{'cb_score': 0.4476019850665217, 'dl_score': 0...",2025-10-02 08:00:52.987256,2025-10-02 08:00:53.472364,apps_score_model_sil,v1,SIL-Instore,M,Mall,android,apps_score_model_sil,SIL,ALL,Test,0.539788,"{""app_cnt_rated_for_3plus_ever"":33,""app_cnt_ed...",2025-10-02 16:00:48,NaT,2025-10
3,2026854,aca20a42-327f-4d0e-b740-75fcf9f709c4,"{'cb_score': 0.29396301987784185, 'dl_score': ...",2025-07-05 05:28:24.718681,2025-07-05 05:28:25.161601,apps_score_model_sil,v1,SIL Competitor,F,Appliance,android,apps_score_model_sil,SIL,ALL,Test,0.384931,"{""app_cnt_rated_for_3plus_ever"":62,""app_cnt_ed...",2025-07-05 13:28:20,2025-07-05 13:35:24,2025-07
4,2148818,fc8ebf76-711c-45c1-a9d0-8aa0a9e9312d,"{'cb_score': 0.42028019411891265, 'dl_score': ...",2025-07-12 03:29:43.654770,2025-07-12 03:29:44.109296,apps_score_model_sil,v1,SIL-Instore,F,Appliance,android,apps_score_model_sil,SIL,ALL,Test,0.483199,"{""app_cnt_rated_for_3plus_ever"":24,""app_cnt_ed...",2025-07-12 11:29:41,NaT,2025-07


In [39]:
df1 = dfd.copy()

### Train

In [40]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - AppsScoreModel' then 'apps_score_model_sil' else modelDisplayName end as modelDisplayName 
     ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - AppsScoreModel', 'apps_score_model_sil')
      ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'apps_score_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Train' Data_selection,
  coalesce(prediction, safe_cast(JSON_VALUE(prediction_clean, "$.combined_score") AS float64)) as score,
 calcFeature calcFeatures,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
 qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base 
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1075642,0c0ff939-0d41-4b5a-933e-dc86498f2746,0.577752,2025-12-23T22:32:14.910582,2025-12-23T22:32:14.910582,apps_score_model_sil,v2,SIL-Instore,M,Appliance,android,apps_score_model_sil,SIL,Trench 2,Train,0.577752,"{""app_cnt_payday_ever"": 1, ""app_cnt_finance_ev...",2025-01-06 15:28:43,2025-01-06 15:33:05,2025-01
1,1254448,b6501739-4b99-44cc-a300-7b5558272f85,0.562780,2025-12-23T22:32:15.510361,2025-12-23T22:32:15.510361,apps_score_model_sil,v2,SIL Competitor,M,Appliance,android,apps_score_model_sil,SIL,Trench 2,Train,0.562780,"{""app_cnt_payday_ever"": 2, ""app_cnt_finance_ev...",2025-11-18 13:32:16,NaT,2025-11
2,1458077,8be1c32a-cf5c-4977-a1f3-da6f9e21a30e,0.546724,2025-12-23T22:17:16.219397,2025-12-23T22:17:16.219397,apps_score_model_sil,v1,SIL-Instore,F,Appliance,android,apps_score_model_sil,SIL,ALL,Train,0.546724,"{""app_cnt_rated_for_3plus_ever"": 3.0, ""app_cnt...",2023-02-24 15:12:41,NaT,2023-02
3,1471170,a0566e14-bccf-48d4-a238-bc7a8b50aaca,0.596381,2025-12-23T22:32:14.699282,2025-12-23T22:32:14.699282,apps_score_model_sil,v2,SIL-Instore,M,Appliance,android,apps_score_model_sil,SIL,Trench 2,Train,0.596381,"{""app_cnt_payday_ever"": 1, ""app_cnt_finance_ev...",2024-11-16 11:58:04,2024-11-16 12:43:08,2024-11
4,1516954,fceb9985-7436-4b14-811a-804a087c6bea,0.413132,2025-12-23T22:17:29.338877,2025-12-23T22:17:29.338877,apps_score_model_sil,v1,SIL-Instore,M,Appliance,android,apps_score_model_sil,SIL,ALL,Train,0.413132,"{""app_cnt_rated_for_3plus_ever"": 27.0, ""app_cn...",2023-01-16 15:18:48,NaT,2023-01


In [41]:
df2 = dfd.copy()

In [42]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (641707, 20)


In [43]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (641707, 20)
The shape of the dataframe after dropping duplicates is: (641707, 20)


In [44]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [45]:
dd.query("""select modelVersionId, Data_selection, count(distinct digitalLoanAccountId) cnt from df_combined group by 1,2;""").to_df()

,modelVersionId,Data_selection,cnt
0,v1,Train,116033
1,v2,Test,55582
2,v2,Train,329229
3,v1,Test,140863


### PSI calculation

In [46]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (55, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-11     score     Training   0.000000         116033             v1   
1   2025-03     score      Monthly   0.075090            373             v1   
2   2025-04     score      Monthly   0.079331            914             v1   
3   2025-05     score      Monthly   0.061396           1389             v1   
4   2025-06     score      Monthly   0.039468          14507             v1   
5   2025-07     score      Monthly   0.041627          20089             v1   
6   2025-08     score      Monthly   0.033829          20563             v1   
7   2025-09     score      Monthly   0.044746          19126             v1   
8   2025-10     score      Monthly   0.051219          22029             v1   


In [47]:
score_output_df

score_output_df['Model_Name'] = 'apps_score_model_sil'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'sil_beta_app_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-11,sil_beta_app_score,Training,0.000000,116033,v1,NA,NA,NA,NA,apps_score_model_sil
1,2025-03,sil_beta_app_score,Monthly,0.075090,373,v1,NA,NA,NA,NA,apps_score_model_sil
2,2025-04,sil_beta_app_score,Monthly,0.079331,914,v1,NA,NA,NA,NA,apps_score_model_sil
3,2025-05,sil_beta_app_score,Monthly,0.061396,1389,v1,NA,NA,NA,NA,apps_score_model_sil
4,2025-06,sil_beta_app_score,Monthly,0.039468,14507,v1,NA,NA,NA,NA,apps_score_model_sil
5,2025-07,sil_beta_app_score,Monthly,0.041627,20089,v1,NA,NA,NA,NA,apps_score_model_sil
6,2025-08,sil_beta_app_score,Monthly,0.033829,20563,v1,NA,NA,NA,NA,apps_score_model_sil
7,2025-09,sil_beta_app_score,Monthly,0.044746,19126,v1,NA,NA,NA,NA,apps_score_model_sil
8,2025-10,sil_beta_app_score,Monthly,0.051219,22029,v1,NA,NA,NA,NA,apps_score_model_sil
9,2025-11,sil_beta_app_score,Monthly,0.055074,25032,v1,NA,NA,NA,NA,apps_score_model_sil


In [48]:
score_output_df.modelVersionId.unique()

array(['v1'], dtype=object)

In [49]:
psi_results = score_output_df.copy()

table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_appscore_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c0b238fe-92e8-4f31-b9be-a9cbdf4bc6c4>

## Beta SIL Demo Score

### Test

In [50]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName ,   
    modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in  ('Beta - DemoScoreModel', 'beta_demo_model_sil')
  ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_demo_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Test' Data_selection,
  -- sil_beta_demo_score
  prediction as score,
 calcFeature calcFeatures,
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid,  modelVersionId  order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
  )
select * from base where lower(new_loan_type) like '%sil%'
;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1080178,1514f860-0f53-4b8d-8fa4-d524592e6349,0.0824249618,2025-11-23 12:27:22.177726,2025-11-23 12:27:22.245782,beta_demo_model_sil,v1,SIL Competitor,F,Mall,ios,beta_demo_model_sil,SIL,ALL,Test,0.0824249618,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2025-11-23 20:27:20,2025-11-23 20:30:28,2025-11
1,1479868,90bdd195-6b2c-4109-8305-a22b53f8b001,0.39907800411120925,2025-12-26 10:14:58.468401,2025-12-26 10:14:58.740710,beta_demo_model_sil,v2,SIL-Instore,M,Appliance,android,beta_demo_model_sil,SIL,Trench 3,Test,0.39907800411120925,"{""demoScoreModel"": {""ln_vas_opted_flag"": ""1"", ...",2025-12-26 18:14:53,2025-12-26 18:21:27,2025-12
2,1546821,be051582-fc10-4d17-b2c8-e061944dfc32,0.2990502894,2025-11-12 06:13:59.629645,2025-11-12 06:13:59.694836,beta_demo_model_sil,v1,SIL Competitor,M,Appliance,android,beta_demo_model_sil,SIL,ALL,Test,0.2990502894,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-11-12 14:13:56,NaT,2025-11
3,1850558,a3785991-1e8f-4c4a-88ed-8b670c50406a,0.1419990673,2025-07-19 09:11:54.601338,2025-07-19 09:11:54.667886,beta_demo_model_sil,v1,SIL Competitor,M,Appliance,ios,beta_demo_model_sil,SIL,ALL,Test,0.1419990673,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-07-19 17:11:52,2025-07-19 17:16:16,2025-07
4,2052789,4e912e3b-db25-4f9e-b5ce-17616e8fe830,0.0680567444,2025-07-26 04:38:34.037401,2025-07-26 04:38:34.098981,beta_demo_model_sil,v1,SIL Competitor,F,Appliance,android,beta_demo_model_sil,SIL,ALL,Test,0.0680567444,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-07-26 12:38:30,2025-07-26 12:41:14,2025-07


In [51]:
df1 = dfd.copy()

### Train

In [52]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - DemoScoreModel' then 'beta_demo_model_sil' else modelDisplayName end as modelDisplayName    
     ,modelVersionId,
        case when trenchCategory is null then 'ALL' 
         when trenchCategory = '' then 'ALL'
    else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in  ('Beta - DemoScoreModel', 'beta_demo_model_sil')
      ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_demo_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Train' Data_selection,
  prediction as score,
 calcFeature calcFeatures,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
 qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base where lower(new_loan_type) like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1045518,4ec0f8af-4296-48a4-84dc-1c560c2736d7,0.145092,2025-12-23T22:18:08.889241,2025-12-23T22:18:08.889241,beta_demo_model_sil,v1,SIL-Instore,M,Appliance,ios,beta_demo_model_sil,SIL,ALL,Train,0.145092,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2024-01-12 11:02:12,2024-01-12 11:09:36,2024-01
1,1087315,9fa761ba-f20e-4ecc-9bf2-5df7ed14d78a,0.287521,2025-12-23T22:18:13.732928,2025-12-23T22:18:13.732928,beta_demo_model_sil,v1,SIL-Instore,M,Appliance,ios,beta_demo_model_sil,SIL,ALL,Train,0.287521,"{""beta_de_ln_vas_opted_flag"": ""0"", ""beta_de_ln...",2023-03-31 10:43:00,2023-03-31 10:52:19,2023-03
2,1223713,c8522857-e34f-4956-9048-cf14accaec63,0.080643,2025-12-23T22:18:11.511240,2025-12-23T22:18:11.511240,beta_demo_model_sil,v1,SIL Repeat,F,Mall,ios,beta_demo_model_sil,SIL,ALL,Train,0.080643,"{""beta_de_ln_vas_opted_flag"": ""1"", ""beta_de_ln...",2025-01-06 18:50:20,2025-01-06 18:54:43,2025-01
3,1225465,60945f33-1bcb-4fea-beec-f0b6bf60ace3,0.418763,2025-12-23T22:34:00.902071,2025-12-23T22:34:00.902071,beta_demo_model_sil,v2,SIL-Instore,F,Appliance,android,beta_demo_model_sil,SIL,Trench 2,Train,0.418763,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",2024-12-09 14:07:02,2024-12-09 14:21:49,2024-12
4,1267752,0cb46c18-75ff-47d2-a0cc-129863b8386e,0.581384,2025-12-23T22:34:00.983911,2025-12-23T22:34:00.983911,beta_demo_model_sil,v2,SIL-Instore,F,Appliance,android,beta_demo_model_sil,SIL,Trench 2,Train,0.581384,"{""ln_vas_opted_flag"": ""1"", ""ln_doc_type_rolled...",2024-12-16 14:35:42,2024-12-16 14:49:36,2024-12


In [53]:
df2 = dfd.copy()

In [54]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (915123, 20)


In [55]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (915123, 20)
The shape of the dataframe after dropping duplicates is: (915123, 20)


In [56]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

### PSI calculation

In [57]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (484, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-11     score     Training   0.000000         321067             v1   
1   2025-03     score      Monthly   0.212490            414             v1   
2   2025-04     score      Monthly   0.271465           1039             v1   
3   2025-05     score      Monthly   0.157399           1561             v1   
4   2025-06     score      Monthly   0.154095          16368             v1   
5   2025-07     score      Monthly   0.187953          22790             v1   
6   2025-08     score      Monthly   0.196011          23270             v1   
7   2025-09     score      Monthly   0.206522          21413             v1   
8   2025-10     score      Monthly   0.280059          24822             v1   

In [58]:
score_output_df

score_output_df['Model_Name'] = 'beta_demo_model_sil'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'sil_beta_demo_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-11,sil_beta_demo_score,Training,0.000000,321067,v1,NA,NA,NA,NA,beta_demo_model_sil
1,2025-03,sil_beta_demo_score,Monthly,0.212490,414,v1,NA,NA,NA,NA,beta_demo_model_sil
2,2025-04,sil_beta_demo_score,Monthly,0.271465,1039,v1,NA,NA,NA,NA,beta_demo_model_sil
3,2025-05,sil_beta_demo_score,Monthly,0.157399,1561,v1,NA,NA,NA,NA,beta_demo_model_sil
4,2025-06,sil_beta_demo_score,Monthly,0.154095,16368,v1,NA,NA,NA,NA,beta_demo_model_sil
...,...,...,...,...,...,...,...,...,...,...,...
479,2025-11,sil_beta_demo_score,Training,0.000000,96,v2,Trench 2,SIL ZERO,Appliance,ios,beta_demo_model_sil
480,2025-11,sil_beta_demo_score,Monthly,20.587815,1,v2,Trench 2,SIL ZERO,Appliance,ios,beta_demo_model_sil
481,2025-12,sil_beta_demo_score,Monthly,6.655938,21,v2,Trench 2,SIL ZERO,Appliance,ios,beta_demo_model_sil
482,2026-01,sil_beta_demo_score,Monthly,8.599668,11,v2,Trench 2,SIL ZERO,Appliance,ios,beta_demo_model_sil


In [59]:
psi_results = score_output_df.copy()

table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_demo_score_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e6aeff59-1199-426e-98e2-ac9cac2293cf>

## Beta SIL STACK Score Model

### Test

In [60]:
sq = """
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - StackScoreModel' then 'beta_stack_model_sil' else modelDisplayName end as modelDisplayName ,   
    modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
  FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
  ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_stack_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Test' Data_selection,
  -- sil_beta_stack_score
  prediction as score,
 calcFeature calcFeatures,
    coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid,  modelVersionId  order by coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) desc) = 1
  )
select * from base
where lower(new_loan_type) like '%sil%'
;
"""
dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1595245,592525a1-38c8-4383-84f1-356268459edd,0.10269050381219676,2025-09-21 07:02:38.575535,2025-09-21 07:02:38.579939,beta_stack_model_sil,v1,SIL Competitor,F,Appliance,android,beta_stack_model_sil,SIL,ALL,Test,0.10269050381219676,"{""s_apps_score"": 0.4952258710665423, ""s_credo_...",2025-09-21 15:02:35,2025-09-21 15:16:49,2025-09
1,1642542,b553ac29-253b-46f0-8070-2db7192c5d55,0.09267829041331731,2025-11-29 07:38:15.069529,2025-11-29 07:38:15.073754,beta_stack_model_sil,v1,SIL-Instore,F,Appliance,android,beta_stack_model_sil,SIL,ALL,Test,0.09267829041331731,"{""s_apps_score"": 0.5933879114156713, ""s_credo_...",2025-11-29 15:38:07,2025-11-29 17:50:07,2025-11
2,1677296,8c4c9bc5-e6cd-445f-bf1e-619ef935022b,0.4779488505411661,2025-12-19 11:10:14.013866,2025-12-19 11:10:14.586669,beta_stack_model_sil,v2,SIL Competitor,F,Appliance,android,beta_stack_model_sil,SIL,Trench 2,Test,0.4779488505411661,"{""stackScoreModel"": {""sb_demo_score"": 0.418002...",2025-12-19 19:10:09,2025-12-19 19:13:29,2025-12
3,1753125,43e5221c-63b7-4fcc-9d4e-563a709ed74f,0.13546228764681115,2025-06-24 12:25:59.856220,2025-06-24 12:25:59.860385,beta_stack_model_sil,v1,SIL-Instore,M,Appliance,android,beta_stack_model_sil,SIL,ALL,Test,0.13546228764681115,"{""s_apps_score"": 0.5169969163853527, ""s_credo_...",2025-06-24 20:25:56,NaT,2025-06
4,1893920,d2c42af7-bca1-4961-8232-5d70563a39e9,0.15930551715949876,2025-11-19 10:24:00.238834,2025-11-19 10:24:00.243461,beta_stack_model_sil,v1,SIL-Instore,F,Appliance,android,beta_stack_model_sil,SIL,ALL,Test,0.15930551715949876,"{""s_apps_score"": 0.5415028011399508, ""s_credo_...",2025-11-19 18:23:56,NaT,2025-11


In [61]:
df1 = dfd.copy()

### Train

In [62]:
sq = """ 
WITH cleaned AS (
  SELECT
    customerId,digitalLoanAccountId,prediction,start_time,end_time,
    case when modelDisplayName = 'Beta - StackScoreModel' then 'beta_stack_model_sil' else modelDisplayName end as modelDisplayName ,   
    modelVersionId,
    case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    REPLACE(REPLACE(cast(prediction as string), "'", '"'), "None", "null") AS prediction_clean
  FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
  WHERE modelDisplayName in ('Beta - StackScoreModel', 'beta_stack_model_sil')
      ),
base as 
(SELECT
  r.customerId,r.digitalLoanAccountId,prediction,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'beta_demo_model_sil' Model_Name,
 'SIL' as product,
  trenchCategory,
  'Train' Data_selection,
  prediction as score,
 calcFeature calcFeatures,
    IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
    loanmaster.disbursementDateTime,
    format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month,
 FROM cleaned r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
 qualify row_number() over (partition by r.customerId,r.digitalLoanAccountId, modelVersionId 
order by   coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) desc) = 1
)
select * from base
where lower(new_loan_type) like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe()
dfd.head()

,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,Model_Name,product,trenchCategory,Data_selection,score,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1132264,6e01fec0-bc68-4615-ad31-98237b115599,0.420498,2025-12-23T22:30:59.734582,2025-12-23T22:30:59.734582,beta_stack_model_sil,v2,SIL-Instore,M,Appliance,ios,beta_demo_model_sil,SIL,Trench 2,Train,0.420498,"{""sb_demo_score"": 0.4702905851215709, ""s_credo...",2024-12-17 11:20:55,NaT,2024-12
1,1192351,8125fbc1-8dc9-45b3-a16d-66e37749a793,0.527965,2025-12-23T22:30:59.412560,2025-12-23T22:30:59.412560,beta_stack_model_sil,v2,SIL-Instore,M,Appliance,android,beta_demo_model_sil,SIL,Trench 2,Train,0.527965,"{""sb_demo_score"": 0.4636337790343312, ""apps_sc...",2025-04-11 12:58:33,NaT,2025-04
2,1291567,bae1063a-5b64-428c-95e5-0038421c77cc,0.269978,2025-12-23T22:19:12.709311,2025-12-23T22:19:12.709311,beta_stack_model_sil,v1,SIL-Instore,M,Appliance,android,beta_demo_model_sil,SIL,ALL,Train,0.269978,"{""apps_score"": 0.526269156213683, ""credo_gen_s...",2023-10-16 15:13:08,2023-10-16 15:18:27,2023-10
3,1391459,c2a59009-c437-4ac5-961c-18ddfd2c9a83,0.505017,2025-12-23T22:30:59.215804,2025-12-23T22:30:59.215804,beta_stack_model_sil,v2,SIL-Instore,F,Appliance,ios,beta_demo_model_sil,SIL,Trench 2,Train,0.505017,"{""sb_demo_score"": 0.5697425226892986, ""s_credo...",2025-01-27 18:17:28,2025-01-27 18:23:59,2025-01
4,1395436,c4a7bf00-b6b3-427f-984a-2b900a19f397,0.175913,2025-12-23T22:19:20.904014,2025-12-23T22:19:20.904014,beta_stack_model_sil,v1,SIL Repeat,F,Mall,android,beta_demo_model_sil,SIL,ALL,Train,0.175913,"{""apps_score"": 0.5175928020293257, ""credo_gen_...",2024-12-21 19:13:52,2024-12-21 19:53:35,2024-12


In [63]:
df2 = dfd.copy()

In [64]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (913029, 20)


In [65]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (913029, 20)
The shape of the dataframe after dropping duplicates is: (913029, 20)


In [66]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

### PSI calculation

In [67]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (484, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-11     score     Training   0.000000         321067             v1   
1   2025-03     score      Monthly   0.135260            414             v1   
2   2025-04     score      Monthly   0.039789           1039             v1   
3   2025-05     score      Monthly   0.042864           1561             v1   
4   2025-06     score      Monthly   0.049583          16368             v1   
5   2025-07     score      Monthly   0.063482          22790             v1   
6   2025-08     score      Monthly   0.058819          23270             v1   
7   2025-09     score      Monthly   0.055673          21413             v1   
8   2025-10     score      Monthly   0.074304          24822             v1   

In [68]:
score_output_df

score_output_df['Model_Name'] = 'beta_stack_model_sil'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'sil_beta_stack_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-11,sil_beta_stack_score,Training,0.000000,321067,v1,NA,NA,NA,NA,beta_stack_model_sil
1,2025-03,sil_beta_stack_score,Monthly,0.135260,414,v1,NA,NA,NA,NA,beta_stack_model_sil
2,2025-04,sil_beta_stack_score,Monthly,0.039789,1039,v1,NA,NA,NA,NA,beta_stack_model_sil
3,2025-05,sil_beta_stack_score,Monthly,0.042864,1561,v1,NA,NA,NA,NA,beta_stack_model_sil
4,2025-06,sil_beta_stack_score,Monthly,0.049583,16368,v1,NA,NA,NA,NA,beta_stack_model_sil
...,...,...,...,...,...,...,...,...,...,...,...
479,2025-11,sil_beta_stack_score,Training,0.000000,95,v2,Trench 2,SIL ZERO,Appliance,ios,beta_stack_model_sil
480,2025-11,sil_beta_stack_score,Monthly,20.552170,1,v2,Trench 2,SIL ZERO,Appliance,ios,beta_stack_model_sil
481,2025-12,sil_beta_stack_score,Monthly,2.130542,21,v2,Trench 2,SIL ZERO,Appliance,ios,beta_stack_model_sil
482,2026-01,sil_beta_stack_score,Monthly,8.377185,11,v2,Trench 2,SIL ZERO,Appliance,ios,beta_stack_model_sil


In [69]:
psi_results = score_output_df.copy()
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_sil_stack_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=91d29f93-0f3b-4451-84e2-7c37792cd50c>

## Cash

### Queries

### Alpha-Cash-CIC-Model (All Trench)

#### Test

In [70]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in('Alpha-Cash-CIC-Model','Alpha Cash CIC Model') then 'cic_model_cash' else modelDisplayName end modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,
 prediction score
 -- aCicScore 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'cic_model_cash' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
""" 

dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID 9a86f71e-9d64-42e3-ba6b-a83b0eb617a7 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1002218,ed21f34d-0b01-4549-8038-50512c4d78d8,0.39857328532206143,2025-11-14 08:02:51.122304,2025-11-14 08:02:51.718793,cic_model_cash,v1,Quick,F,not applicable,android,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 2256.0,...",2025-11-14 16:02:19,2025-11-15 20:01:29,2025-11
1,1014767,6ae06a14-c10c-4cf2-acf9-ffdd3a5c58e0,0.6113682602132733,2025-12-28 10:36:58.135356,2025-12-28 10:36:58.230342,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 1850, ""...",2025-12-28 18:36:41,NaT,2025-12
2,1065947,d406c4b5-c921-415e-8d8a-d3dd56798649,0.4740446908971318,2025-11-30 00:31:50.152539,2025-11-30 00:31:50.988222,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 3,Test,"{""cic_max_age_all_contracts_snapshot"": 1569.0,...",2025-11-30 08:31:19,2025-11-30 20:13:57,2025-11
3,1094172,906eef40-9bf1-41fb-b804-d6116e25da8a,0.5669626982784635,2025-12-15 10:17:23.570662,2025-12-15 10:17:23.823755,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 876, ""c...",2025-12-15 18:16:57,NaT,2025-12
4,1108284,c0de2a67-1978-4c4b-b1fd-8682926cebca,0.45183722236950935,2025-12-15 10:08:20.972220,2025-12-15 10:08:21.170433,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 3103.0,...",2025-12-15 18:07:53,NaT,2025-12


In [71]:
df1 = dfd.copy()

#### Train

In [72]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in('Alpha-Cash-CIC-Model','Alpha Cash CIC Model') then 'cic_model_cash' end modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-CIC-Model','Alpha Cash CIC Model','cic_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,
 prediction score 
 ,start_time,end_time,
  r.modelDisplayName,
  r.modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'cic_model_cash' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


Job ID 1fa31b89-3e47-4baa-bb45-8e8ce61d637e successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1121459,baa15a3b-ef2f-44cc-8677-ce669208dae6,0.485187,2025-12-23T22:23:28.011196,2025-12-23T22:23:28.011196,cic_model_cash,v1,Quick,F,not applicable,android,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 2823.0, ""ra...",2024-09-02 08:57:29,NaT,2024-09
1,1232988,794e5133-0196-49ef-b138-b61db158fe95,0.503522,2025-12-23T22:23:29.449631,2025-12-23T22:23:29.449631,cic_model_cash,v1,Quick,F,not applicable,android,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 1326.0, ""ra...",2024-09-30 22:40:39,2024-10-02 19:27:22,2024-09
2,1308551,dc64e623-86a5-4516-bef2-4c2d8ce589d1,0.500756,2025-12-23T22:23:40.903985,2025-12-23T22:23:40.903985,cic_model_cash,v1,Quick,F,not applicable,android,Cash,cic_model_cash,Trench 3,Train,"{""max_age_all_contracts_snapshot"": 789.0, ""rat...",2024-09-03 16:15:48,NaT,2024-09
3,1392027,3f88985e-dab3-4c1e-ab2c-a557fe3c4c70,0.453358,2025-12-23T22:23:42.670870,2025-12-23T22:23:42.670870,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 3,Train,"{""max_age_all_contracts_snapshot"": 1933.0, ""ra...",2024-09-17 22:03:32,2024-09-17 22:07:28,2024-09
4,1427266,3d608695-1173-477d-988e-c0f992608d0c,0.482426,2025-12-23T22:23:29.054510,2025-12-23T22:23:29.054510,cic_model_cash,v1,Quick,M,not applicable,android,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 739.0, ""rat...",2024-09-02 18:51:26,2024-09-02 19:17:31,2024-09


In [73]:
df2 = dfd.copy()

In [74]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (224532, 19)


In [75]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (224532, 19)
The shape of the dataframe after dropping duplicates is: (224532, 19)


In [76]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [77]:
df_combined.sample(10)

,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
178128,3435942,115decf7-6e3a-4609-a2a8-b492ef33fec3,0.525444,2025-12-23T22:23:15.282419,2025-12-23T22:23:15.282419,cic_model_cash,v1,Quick,F,not applicable,android,Cash,cic_model_cash,Trench 1,Train,"{""max_age_all_contracts_snapshot"": 366.0, ""rat...",2025-05-25 13:14:59,NaT,2025-05
9377,3989718,2c96aad2-86f4-40a3-8586-de719186f1e5,0.725422,2026-01-09 15:41:47.309184,2026-01-09 15:41:47.415607,cic_model_cash,v1,Quick,M,not applicable,android,Cash,cic_model_cash,Trench 1,Test,"{""cic_max_age_all_contracts_snapshot"": 1041, ""...",2026-01-09 23:41:31,NaT,2026-01
100519,2822735,bc62571c-e34c-4945-84fe-15b52c523af5,0.647759,2025-12-23T22:23:26.880470,2025-12-23T22:23:26.880470,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 2273.0, ""ra...",2024-10-13 17:51:56,NaT,2024-10
149023,3270084,9329a9af-f7ba-4997-812e-821142bb453f,0.492525,2025-12-23T22:23:11.517307,2025-12-23T22:23:11.517307,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 1,Train,"{""max_age_all_contracts_snapshot"": 99.0, ""rati...",2025-02-18 20:17:53,NaT,2025-02
35657,2543669,05410c02-8c0d-4cc7-b58d-9cb94cb4df19,0.492289,2025-10-09 05:55:31.426942,2025-10-09 05:55:31.544465,cic_model_cash,v1,Quick,F,not applicable,ios,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 2172.0,...",2025-10-09 13:55:17,2025-11-04 13:35:24,2025-10
201087,1368072,0c20ee64-b0a4-4ad6-b747-2f1c20652bb9,0.695610,2025-12-23T22:23:27.016776,2025-12-23T22:23:27.016776,cic_model_cash,v1,Quick,M,not applicable,android,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 996.0, ""rat...",2025-07-20 22:31:18,NaT,2025-07
76278,3951936,dbc16501-6abf-4faa-82a3-b1fca2ef1327,0.568835,2025-12-30 00:50:45.857231,2025-12-30 00:50:46.034060,cic_model_cash,v1,Quick,F,not applicable,android,Cash,cic_model_cash,Trench 1,Test,"{""cic_max_age_all_contracts_snapshot"": 275, ""c...",2025-12-30 08:50:32,NaT,2025-12
164619,2067119,7a4d7d96-bcc6-4e84-830c-01c0336d048a,0.754098,2025-12-23T22:23:32.919705,2025-12-23T22:23:32.919705,cic_model_cash,v1,Quick,M,not applicable,android,Cash,cic_model_cash,Trench 2,Train,"{""max_age_all_contracts_snapshot"": 179.0, ""rat...",2025-04-06 16:05:24,NaT,2025-04
175227,2847959,cbf97a3d-d103-43ed-880b-e83a574c70f8,0.419928,2025-12-23T22:23:41.932424,2025-12-23T22:23:41.932424,cic_model_cash,v1,Quick,M,not applicable,ios,Cash,cic_model_cash,Trench 3,Train,"{""max_age_all_contracts_snapshot"": 234.0, ""rat...",2025-05-17 16:10:18,2025-05-17 16:40:53,2025-05
26910,2699083,36ec621b-7be1-40aa-9e9d-ec2d1c92d2e1,0.767395,2025-10-13 00:46:32.808772,2025-10-13 00:46:32.933068,cic_model_cash,v1,Quick,F,not applicable,android,Cash,cic_model_cash,Trench 2,Test,"{""cic_max_age_all_contracts_snapshot"": 351, ""c...",2025-10-13 08:46:21,NaT,2025-10


### PSI calculation

In [78]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (112, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-09     score     Training   0.000000         145746             v1   
1   2025-09     score      Monthly   0.071033            134             v1   
2   2025-10     score      Monthly   0.004982           9210             v1   
3   2025-11     score      Monthly   0.021048          18803             v1   
4   2025-12     score      Monthly   0.073764          29260             v1   
5   2026-01     score      Monthly   0.017216          19254             v1   
6   2026-02     score      Monthly   0.018879           2125             v1   
7   2025-09     score     Training   0.000000          48479             v1   
8   2025-09     score      Monthly   2.194329             31             v1   

In [79]:
score_output_df
score_output_df['Model_Name'] = 'cic_model_cash'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'cic_model_cash_score')
score_output_df

,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-09,cic_model_cash_score,Training,0.000000,145746,v1,NA,NA,NA,NA,cic_model_cash
1,2025-09,cic_model_cash_score,Monthly,0.071033,134,v1,NA,NA,NA,NA,cic_model_cash
2,2025-10,cic_model_cash_score,Monthly,0.004982,9210,v1,NA,NA,NA,NA,cic_model_cash
3,2025-11,cic_model_cash_score,Monthly,0.021048,18803,v1,NA,NA,NA,NA,cic_model_cash
4,2025-12,cic_model_cash_score,Monthly,0.073764,29260,v1,NA,NA,NA,NA,cic_model_cash
...,...,...,...,...,...,...,...,...,...,...,...
107,2025-10,cic_model_cash_score,Monthly,0.040182,332,v1,Trench 3,Quick,not applicable,ios,cic_model_cash
108,2025-11,cic_model_cash_score,Monthly,0.055762,363,v1,Trench 3,Quick,not applicable,ios,cic_model_cash
109,2025-12,cic_model_cash_score,Monthly,0.066168,423,v1,Trench 3,Quick,not applicable,ios,cic_model_cash
110,2026-01,cic_model_cash_score,Monthly,0.108791,383,v1,Trench 3,Quick,not applicable,ios,cic_model_cash


In [80]:
psi_results = score_output_df.copy()
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_cic_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=399f8231-736d-4e24-99d7-fc870b846dbb>

### Alpha-Cash-Stack-Model

#### Test

In [81]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,
case when  modelDisplayName in ('Alpha-Cash-Stack-Model') then 'alpha_stack_model_cash' else modelDisplayName end as modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'alpha_stack_model_cash' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  prediction score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""

dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID e20e4bf1-89ab-4ce4-88fa-e6ed2af55f4d successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
0,1017462,f53b785b-a112-4404-bbbe-2ab4f7c359f5,2025-11-12 11:42:16.582906,2025-11-12 11:42:16.745202,alpha_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": null, ""c_demo_score"": 0.4585690...",0.22838469082766744,2025-11-12 19:41:58,2025-11-14 00:35:04,2025-11
1,1037029,b3afd63d-d2db-42dd-9371-ecca77c7ceed,2025-11-26 02:40:04.418079,2025-11-26 02:40:04.518071,alpha_stack_model_cash,v1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": 0.4312772692633179, ""c_demo_sco...",0.44305961879263495,2025-11-26 10:39:52,NaT,2025-11
2,1042179,6145328c-e1c7-47e9-8a7a-0caa316fccab,2025-12-14 08:15:04.264188,2025-12-14 08:15:04.386257,alpha_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 3,Test,"{""apps_score"": null, ""c_demo_score"": 0.5704856...",0.6047458064085982,2025-12-14 16:14:39,NaT,2025-12
3,1046073,59ae2d30-eefc-4ba5-a725-0248964c51f7,2026-01-10 11:03:44.832233,2026-01-10 11:03:45.041757,alpha_stack_model_cash,v1.1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": null, ""c_demo_score"": 0.4420419...",0.1426367366633122,2026-01-10 12:02:01,NaT,2026-01
4,1051556,a28b3d58-d9b1-4983-aa92-ccfa47cdcca4,2025-12-28 00:41:42.605082,2025-12-28 00:41:42.942963,alpha_stack_model_cash,v1.1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": null, ""c_demo_score"": 0.5275841...",0.2802795524297271,2025-12-28 08:41:16,2025-12-28 08:50:26,2025-12


In [82]:
df1 = dfd.copy()

#### Train

In [83]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,
case when  modelDisplayName in ('Alpha-Cash-Stack-Model') then 'alpha_stack_model_cash' else modelDisplayName end as modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Alpha-Cash-Stack-Model', 'alpha_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction score 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type ,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'alpha_stack_model_cash' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'

"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


Job ID 9a18682c-4ced-404f-89ca-ba0d8807c5c2 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1004026,311a0155-203b-49ec-bc2a-0d5088b6dc8f,0.056928,2025-12-23T22:22:48.947548,2025-12-23T22:22:48.947548,alpha_stack_model_cash,v1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.36585017428292427, ""apps_scor...",2024-10-05 00:03:44,2024-10-05 07:08:04,2024-10
1,1021940,0d2ddb0e-c108-45ec-a752-3aa53eaaa1f7,0.074307,2025-12-23T22:22:48.948547,2025-12-23T22:22:48.948547,alpha_stack_model_cash,v1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.5275419822618007, ""apps_score...",2024-10-07 11:40:51,2024-10-07 11:49:51,2024-10
2,1145884,6409a9ef-6dc9-436f-86a1-bbc14c7a6e4d,0.162858,2025-12-23T22:22:58.915756,2025-12-23T22:22:58.915756,alpha_stack_model_cash,v1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 3,Train,"{""demo_score"": 0.17277600500728285, ""apps_scor...",2024-10-01 09:04:19,2024-10-02 11:16:50,2024-10
3,1308667,077e46f3-5161-4ada-a75d-a6be90ac33e2,0.502167,2025-12-23T22:22:52.298204,2025-12-23T22:22:52.298204,alpha_stack_model_cash,v1,Quick,M,not applicable,ios,Cash,alpha_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.5799678061411861, ""credo_scor...",2024-10-26 15:48:51,NaT,2024-10
4,1360274,0d2f9c8e-6d23-4091-becc-cbaa844425f9,0.465288,2025-12-23T22:22:52.127250,2025-12-23T22:22:52.127250,alpha_stack_model_cash,v1,Quick,M,not applicable,ios,Cash,alpha_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.6064638983421778, ""credo_scor...",2024-10-21 01:39:37,NaT,2024-10


In [84]:
df2 = dfd.copy()

In [85]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (207844, 19)


In [86]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (207844, 19)
The shape of the dataframe after dropping duplicates is: (207844, 19)


In [87]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [88]:
df_combined.sample(10)

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
60125,3059979,12096826-7957-4513-8231-cfbd05e74143,2026-01-28 02:10:05.638237,2026-01-28 02:10:05.956696,alpha_stack_model_cash,v1.1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": null, ""c_demo_score"": 0.5857952...",0.496245,2026-01-28 10:09:41,NaT,2026-01
75363,4007116,0dae8191-1afb-4792-aa2a-d5fa1ddba160,2026-01-14 11:46:48.158966,2026-01-14 11:46:48.307081,alpha_stack_model_cash,v1.1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 1,Test,"{""apps_score"": 0.5286986216991907, ""c_demo_sco...",0.607258,2026-01-14 19:46:28,NaT,2026-01
205313,3667119,3765ae90-5bf8-43bf-a538-cfeae704e785,2025-12-23T22:22:39.907434,2025-12-23T22:22:39.907434,alpha_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.4825711136146396, ""credo_scor...",0.325032,2025-09-06 10:28:08,NaT,2025-09
1865,3595413,d75141de-233c-4300-8e32-88182d0f75c0,2025-11-13 07:54:28.998138,2025-11-13 07:54:29.095785,alpha_stack_model_cash,v1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Test,"{""apps_score"": 0.5483874997913365, ""c_demo_sco...",0.639947,2025-11-13 15:54:09,NaT,2025-11
57584,4069220,eb77e77f-bc5a-46d1-8fce-3d7c20cfb0c3,2026-02-01 23:44:31.498918,2026-02-01 23:44:31.766684,alpha_stack_model_cash,v1.1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 1,Test,"{""apps_score"": null, ""c_demo_score"": 0.5772055...",0.456719,2026-02-02 07:44:00,2026-02-02 11:34:50,2026-02
91706,3058020,867eadc4-63ef-418f-82f5-d2580bd53f9e,2025-12-23T22:22:35.498424,2025-12-23T22:22:35.498424,alpha_stack_model_cash,v1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.43323989259458195, ""apps_scor...",0.240447,2024-11-27 01:21:39,2024-12-02 14:54:44,2024-11
148527,1951019,c78bad12-6da4-4c3d-b0f9-ccef31f43882,2025-12-23T22:22:49.257302,2025-12-23T22:22:49.257302,alpha_stack_model_cash,v1,Quick,M,not applicable,android,Cash,alpha_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.624620801488216, ""apps_score""...",0.630554,2025-04-16 23:19:41,NaT,2025-04
14869,3978980,23057057-bf82-4607-9a2b-69a7afecdb13,2026-01-07 05:40:45.479314,2026-01-07 05:40:45.806065,alpha_stack_model_cash,v1.1,Quick,F,not applicable,android,Cash,alpha_stack_model_cash,Trench 1,Test,"{""apps_score"": 0.524643009747184, ""c_demo_scor...",0.409139,2026-01-07 13:40:12,2026-01-07 13:43:48,2026-01
52701,3901937,4c091e12-6ed2-446d-9cc6-393f435bcb00,2025-12-16 02:59:26.931822,2025-12-16 02:59:27.083780,alpha_stack_model_cash,v1.1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 1,Test,"{""apps_score"": null, ""c_demo_score"": 0.4433377...",0.353724,2025-12-16 10:59:12,NaT,2025-12
185639,2967367,f4bdfcb7-f1ec-4942-9afb-08b056545384,2025-12-23T22:23:00.170100,2025-12-23T22:23:00.170100,alpha_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,alpha_stack_model_cash,Trench 3,Train,"{""demo_score"": 0.5874747609494186, ""credo_scor...",0.490194,2025-07-17 05:48:14,2025-07-17 07:31:16,2025-07


### PSI calculation

In [89]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (92, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-09     score     Training   0.000000         129058             v1   
1   2025-09     score      Monthly   0.573402            134             v1   
2   2025-10     score      Monthly   0.133387           9210             v1   
3   2025-11     score      Monthly   0.135991          18803             v1   
4   2025-12     score      Monthly   0.138993          11761             v1   
5   2026-01     score      Monthly   2.768859           1023             v1   
6   2026-02     score      Monthly   2.405244            149             v1   
7   2025-09     score     Training   0.000000          42982             v1   
8   2025-09     score      Monthly   4.296736             31             v1   


In [90]:
print(f"The shape of the score_output_df is {score_output_df.shape}")
score_output_df['Model_Name'] = 'alpha_stack_model_cash'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'alpha_stack_model_cash_score')
score_output_df

The shape of the score_output_df is (92, 10)


,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-09,alpha_stack_model_cash_score,Training,0.000000,129058,v1,NA,NA,NA,NA,alpha_stack_model_cash
1,2025-09,alpha_stack_model_cash_score,Monthly,0.573402,134,v1,NA,NA,NA,NA,alpha_stack_model_cash
2,2025-10,alpha_stack_model_cash_score,Monthly,0.133387,9210,v1,NA,NA,NA,NA,alpha_stack_model_cash
3,2025-11,alpha_stack_model_cash_score,Monthly,0.135991,18803,v1,NA,NA,NA,NA,alpha_stack_model_cash
4,2025-12,alpha_stack_model_cash_score,Monthly,0.138993,11761,v1,NA,NA,NA,NA,alpha_stack_model_cash
5,2026-01,alpha_stack_model_cash_score,Monthly,2.768859,1023,v1,NA,NA,NA,NA,alpha_stack_model_cash
6,2026-02,alpha_stack_model_cash_score,Monthly,2.405244,149,v1,NA,NA,NA,NA,alpha_stack_model_cash
7,2025-09,alpha_stack_model_cash_score,Training,0.000000,42982,v1,Trench 2,NA,NA,NA,alpha_stack_model_cash
8,2025-09,alpha_stack_model_cash_score,Monthly,4.296736,31,v1,Trench 2,NA,NA,NA,alpha_stack_model_cash
9,2025-10,alpha_stack_model_cash_score,Monthly,0.340872,2355,v1,Trench 2,NA,NA,NA,alpha_stack_model_cash


In [91]:
psi_results = score_output_df.copy()
# Upload to BigQuery
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.alpha_cash_stack_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result()  # Wait for the job to complete


C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=568d2fe5-4ebc-48fe-a65d-586a5f8eb839>

### Beta-Cash-Demo-Model

#### Test

In [92]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in ('Beta-Cash-Demo-Model') then 'beta_demo_model_cash' else modelDisplayName end modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId
 ,start_time,end_time,  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'beta_demo_model_cash' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  prediction score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""
dfd= client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID a3b22575-8eb3-43f9-b750-aac036087409 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
0,1029958,282d058e-12cb-4436-9929-96bb8fc2a4d3,2026-01-02 03:18:14.988537,2026-01-02 03:18:15.706739,beta_demo_model_cash,v1,Quick,M,not applicable,ios,Cash,beta_demo_model_cash,Trench 3,Test,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",0.2912303101971965,2026-01-02 11:18:08,2026-01-14 12:55:31,2026-01
1,1035705,589e4a62-1d99-4b48-a2a1-af33e3eebd40,2025-12-27 03:22:50.797962,2025-12-27 03:22:51.347482,beta_demo_model_cash,v1,Quick,M,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",0.48692895234074496,2025-12-27 11:22:46,2025-12-27 22:26:10,2025-12
2,1039274,9c988705-f3f6-4bce-a03f-8a5d53409351,2025-11-18 01:22:30.639268,2025-11-18 01:22:31.196075,beta_demo_model_cash,v1,Quick,M,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",0.44759050673692896,2025-11-18 09:22:27,NaT,2025-11
3,1048719,cb5e212d-2b38-4834-b4fc-f91d66266a37,2025-10-13 21:07:43.861219,2025-10-13 21:07:44.649830,beta_demo_model_cash,v1,Quick,M,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",0.5930810995480369,2025-10-14 05:07:40,2025-10-14 14:45:58,2025-10
4,1049753,ea86e003-f4c1-4c7e-95c1-8aea2496e7e1,2025-12-16 03:47:58.736677,2025-12-16 03:47:59.743262,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",0.3277276579402258,2025-12-16 11:47:54,2025-12-18 10:48:42,2025-12


In [93]:
df1 = dfd.copy()

#### Train

In [94]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in ('Beta-Cash-Demo-Model') then 'beta_demo_model_cash' else modelDisplayName end modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Demo-Model', 'beta_demo_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction 
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'beta_demo_model_cash' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  prediction score,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()


Job ID c3b64c12-1118-42d5-965f-d667051e2785 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
0,1000979,34de2fdd-fa6f-4e17-b920-5db5616cd7ac,0.454334,2025-12-23T22:24:59.444348,2025-12-23T22:24:59.444348,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",0.454334,2025-02-15 17:21:19,NaT,2025-02
1,1002586,639b8ff3-fe2e-4e64-8cec-2effbd99e617,0.638047,2025-12-23T22:25:01.837923,2025-12-23T22:25:01.837923,beta_demo_model_cash,v1,Quick,M,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",0.638047,2025-02-26 15:07:38,NaT,2025-02
2,1006260,1b1401d3-54e2-49c2-83a6-7ae864c6d2c1,0.469934,2025-12-23T22:25:07.054284,2025-12-23T22:25:07.054284,beta_demo_model_cash,v1,Quick,M,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",0.469934,2025-09-05 00:45:35,2025-09-19 20:34:53,2025-09
3,1014631,306bb222-52cb-4049-882d-25336dcc5f20,0.692974,2025-12-23T22:24:58.343232,2025-12-23T22:24:58.343232,beta_demo_model_cash,v1,Quick,M,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",0.692974,2025-03-19 22:50:42,NaT,2025-03
4,1021322,d03b8797-f6b2-4f58-8b60-2bd7c3fa36a1,0.459456,2025-12-23T22:25:02.555706,2025-12-23T22:25:02.555706,beta_demo_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",0.459456,2024-11-04 18:16:23,NaT,2024-11


In [95]:
df2 = dfd.copy()

In [96]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (765381, 20)


In [97]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (765381, 20)
The shape of the dataframe after dropping duplicates is: (765381, 20)


In [98]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [99]:
df_combined.sample(10)

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month,prediction
83254,3858357,4e1a49ed-e2c3-41ee-8fe0-41badfc8abd1,2025-12-04 02:26:08.018929,2025-12-04 02:26:08.295423,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Test,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",0.580652,2025-12-04 10:26:04,NaT,2025-12,NaN
100295,4007366,5393a68d-16db-450a-907b-922efe61dfde,2026-01-14 14:16:13.973805,2026-01-14 14:16:14.576162,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Test,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",0.640604,2026-01-14 22:16:09,NaT,2026-01,NaN
722701,2981006,20973fcd-5ea0-4f94-85d2-2601d01273ac,2025-12-23T22:24:18.634561,2025-12-23T22:24:18.634561,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Train,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",0.485922,2024-10-28 22:12:54,NaT,2024-10,0.485922
32194,4070972,49eb38fc-f6ea-4713-a15b-c9b891f549ee,2026-02-02 09:31:41.283247,2026-02-02 09:31:41.879094,beta_demo_model_cash,v1,Quick,M,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Test,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",0.627327,2026-02-02 17:31:36,NaT,2026-02,NaN
376487,3533203,a1b553de-6770-486f-93d9-6b30849a1618,2025-12-23T22:23:56.801359,2025-12-23T22:23:56.801359,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Train,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",0.622590,2025-07-03 08:02:44,NaT,2025-07,0.622590
742169,2948903,ea1125b8-3acc-462e-abdf-d0add1044e22,2025-12-23T22:24:14.172632,2025-12-23T22:24:14.172632,beta_demo_model_cash,v1,Quick,M,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Train,"{""ln_vas_opted_flag"": ""1"", ""ln_self_dec_income...",0.634221,2024-10-17 16:17:21,NaT,2024-10,0.634221
221394,4051938,d12a00bb-1bf0-4380-bb43-9fe554dd770d,2026-01-27 19:33:38.667774,2026-01-27 19:33:39.268255,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Test,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",0.554170,2026-01-28 03:33:33,NaT,2026-01,NaN
120851,3740782,cff1efeb-8c97-4555-9c0c-a290d82333d9,2025-10-12 16:24:53.094909,2025-10-12 16:24:53.350501,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 1,Test,"{""ln_vas_opted_flag"": 1, ""ln_self_dec_income"":...",0.619963,2025-10-13 00:24:50,NaT,2025-10,NaN
555023,2856624,7adc5048-6185-4fe9-ae62-65c130252f80,2025-12-23T22:25:06.785861,2025-12-23T22:25:06.785861,beta_demo_model_cash,v1,Quick,F,not applicable,android,Cash,beta_demo_model_cash,Trench 2,Train,"{""ln_vas_opted_flag"": ""0"", ""ln_self_dec_income...",0.563160,2025-07-04 19:11:20,NaT,2025-07,0.563160
188951,2663232,659ab382-36f4-408c-afd5-e9587961e2a1,2025-11-01 12:33:03.068137,2025-11-01 12:33:03.840645,beta_demo_model_cash,v1,Quick,M,not applicable,ios,Cash,beta_demo_model_cash,Trench 2,Test,"{""ln_vas_opted_flag"": 0, ""ln_self_dec_income"":...",0.378762,2025-11-01 20:33:00,NaT,2025-11,NaN


### PSI calculation

In [100]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (117, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-10     score     Training   0.000000         491421             v1   
1   2025-02     score      Monthly  20.713669              1             v1   
2   2025-09     score      Monthly   0.020071            803             v1   
3   2025-10     score      Monthly   0.003881          35272             v1   
4   2025-11     score      Monthly   0.016022          58553             v1   
5   2025-12     score      Monthly   0.021845          96115             v1   
6   2026-01     score      Monthly   0.004769          75428             v1   
7   2026-02     score      Monthly   0.008798           7788             v1   
8   2025-10     score     Training   0.000000         324441             v1   

In [101]:
print(f"The shape of the score_output_df is {score_output_df.shape}")
score_output_df['Model_Name'] = 'beta_demo_model_cash'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'beta_demo_model_cash_score')
score_output_df

The shape of the score_output_df is (117, 10)


,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-10,beta_demo_model_cash_score,Training,0.000000,491421,v1,NA,NA,NA,NA,beta_demo_model_cash
1,2025-02,beta_demo_model_cash_score,Monthly,20.713669,1,v1,NA,NA,NA,NA,beta_demo_model_cash
2,2025-09,beta_demo_model_cash_score,Monthly,0.020071,803,v1,NA,NA,NA,NA,beta_demo_model_cash
3,2025-10,beta_demo_model_cash_score,Monthly,0.003881,35272,v1,NA,NA,NA,NA,beta_demo_model_cash
4,2025-11,beta_demo_model_cash_score,Monthly,0.016022,58553,v1,NA,NA,NA,NA,beta_demo_model_cash
...,...,...,...,...,...,...,...,...,...,...,...
112,2025-10,beta_demo_model_cash_score,Monthly,0.026257,813,v1,Trench 3,Quick,not applicable,ios,beta_demo_model_cash
113,2025-11,beta_demo_model_cash_score,Monthly,0.013132,1000,v1,Trench 3,Quick,not applicable,ios,beta_demo_model_cash
114,2025-12,beta_demo_model_cash_score,Monthly,0.039445,1478,v1,Trench 3,Quick,not applicable,ios,beta_demo_model_cash
115,2026-01,beta_demo_model_cash_score,Monthly,0.039550,933,v1,Trench 3,Quick,not applicable,ios,beta_demo_model_cash


In [102]:
psi_results = score_output_df.copy()
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_demo_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result() 

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=9781a69d-dfc3-4ffb-8c02-2aa5b2488af8>

### Beta-Cash-AppScore-Model

#### Test

In [103]:
sq = """WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in ('Beta-Cash-AppScore-Model') then 'apps_score_cash' else modelDisplayName end as modelDisplayName,
modelVersionId,start_time,end_time,
prediction,
case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type ,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'apps_score_cash' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  SAFE_CAST(JSON_VALUE(r.prediction_clean, "$.combined_score") AS Float64) AS score,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID 22be120d-7e2b-4d0f-a521-2d241764121d successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
0,1002218,ed21f34d-0b01-4549-8038-50512c4d78d8,2025-11-14 08:02:27.080302,2025-11-14 08:02:27.818765,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",0.460862,2025-11-14 16:02:19,2025-11-15 20:01:29,2025-11
1,1048996,ac994292-0285-4388-b2c8-e0245dfe252d,2025-11-14 14:12:06.771066,2025-11-14 14:12:07.630836,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",0.390034,2025-11-14 22:12:04,NaT,2025-11
2,1118368,569d3863-0f72-409a-bb67-732ea40b8343,2025-10-23 11:56:57.939050,2025-10-23 11:56:58.833302,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 3,Test,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",0.499402,2025-10-23 19:56:53,2025-10-24 08:55:19,2025-10
3,1143824,7e7ec78c-3845-4a03-a269-279721c78bf0,2025-12-09 05:47:08.673294,2025-12-09 05:47:09.446207,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 3, ""app_cn...",0.495429,2025-12-09 13:47:05,NaT,2025-12
4,1184055,3f2c286a-f193-415b-9939-f6f43a6b606f,2025-12-11 08:36:07.421745,2025-12-11 08:36:08.106937,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",0.522536,2025-12-11 16:36:04,NaT,2025-12


In [104]:
df1 = dfd.copy()

#### Train

In [105]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in  ('Beta-Cash-AppScore-Model') then 'apps_score_cash' else modelDisplayName end as modelDisplayName,
modelVersionId,start_time,end_time,prediction,
   case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in  ('Beta-Cash-AppScore-Model', 'apps_score_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type ,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'apps_score_cash' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  prediction score,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID a950091f-a3a7-4bd3-8d72-bdccf810f43b successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
0,1004026,311a0155-203b-49ec-bc2a-0d5088b6dc8f,2025-12-23T22:27:52.617647,2025-12-23T22:27:52.617647,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 2,Train,"{""beta_cash_app_score"": 0.37570252126280135, ""...",0.375703,2024-10-05 00:03:44,2024-10-05 07:08:04,2024-10
1,1010418,ac25f4f5-d340-4565-8493-4d6e6a169255,2025-12-23T22:27:48.105809,2025-12-23T22:27:48.105809,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Train,"{""beta_cash_app_score"": 0.4581980216314372, ""a...",0.458198,2024-11-08 05:20:57,2024-11-08 09:03:52,2024-11
2,1012276,0646eac1-3910-45ac-a473-009486a68cc0,2025-12-23T22:28:08.018064,2025-12-23T22:28:08.018064,apps_score_cash,v1,Flex-up,M,not applicable,android,Cash,apps_score_cash,Trench 3,Train,"{""beta_cash_app_score"": 0.4837266612850275, ""a...",0.483727,2024-09-23 09:53:34,2024-09-23 09:56:57,2024-09
3,1013152,5756e61a-2a2d-4cbc-9dfa-b44435096484,2025-12-23T22:28:13.098602,2025-12-23T22:28:13.098602,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 3,Train,"{""beta_cash_app_score"": 0.49565420092814827, ""...",0.495654,2025-07-31 18:19:31,NaT,2025-07
4,1021940,0d2ddb0e-c108-45ec-a752-3aa53eaaa1f7,2025-12-23T22:27:52.987468,2025-12-23T22:27:52.987468,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 2,Train,"{""beta_cash_app_score"": 0.2963325925509856, ""a...",0.296333,2024-10-07 11:40:51,2024-10-07 11:49:51,2024-10


In [106]:
df2 = dfd.copy()

In [107]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (609916, 19)


In [108]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (609916, 19)
The shape of the dataframe after dropping duplicates is: (609916, 19)


In [109]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [110]:
df_combined.sample(10)

,customerId,digitalLoanAccountId,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,score,appln_submit_datetime,disbursementDateTime,Application_month
44363,3966129,f7552413-8c61-4922-9c29-55fa6ee59d4e,2026-01-03 07:56:48.135436,2026-01-03 07:56:48.776860,apps_score_cash,v1.1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 1,Test,"{""app_cnt_rated_for_18plus_ever"": 0, ""app_cnt_...",0.606866,2026-01-03 15:56:43,NaT,2026-01
517635,2859894,a8726ff0-cffd-447b-9cbe-e8f285193561,2025-12-23T22:27:23.548749,2025-12-23T22:27:23.548749,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 1,Train,"{""beta_cash_app_score"": 0.6419553706156942, ""a...",0.641955,2024-09-19 11:33:36,NaT,2024-09
559339,3213269,2f977ebe-ef20-4538-b24b-2f1921847c5b,2025-12-23T22:27:12.915723,2025-12-23T22:27:12.915723,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 1,Train,"{""beta_cash_app_score"": 0.6183919737776069, ""a...",0.618392,2025-01-25 10:10:25,NaT,2025-01
352712,3306124,07dbc9af-4caa-424c-8d50-8f6f4c342f53,2025-12-23T22:26:58.672896,2025-12-23T22:26:58.672896,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 1,Train,"{""beta_cash_app_score"": 0.6448012235189255, ""a...",0.644801,2025-03-07 12:22:41,NaT,2025-03
515604,3442053,16880c58-1da5-4e3e-baeb-4a427fdf4337,2025-12-23T22:27:04.939306,2025-12-23T22:27:04.939306,apps_score_cash,v1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 1,Train,"{""beta_cash_app_score"": 0.5420504944792746, ""a...",0.542050,2025-05-17 15:14:42,NaT,2025-05
123120,3824687,ce5a51d2-9e9a-4a70-b0db-cb21b0ef6572,2025-11-21 04:18:22.727897,2025-11-21 04:18:23.029082,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 1,Test,"{""app_cnt_health_and_fitness_ever"": 1, ""app_cn...",0.697098,2025-11-21 12:18:19,NaT,2025-11
194355,3576489,82d16411-4808-4400-b5e0-d3ecdde1ca50,2025-11-19 00:19:11.191807,2025-11-19 00:19:11.996125,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_health_and_fitness_ever"": 0, ""app_cn...",0.635408,2025-11-19 08:19:08,NaT,2025-11
83118,4022422,d01c12b2-67c1-4999-aa90-1f2c42eafbb9,2026-01-19 02:10:26.460312,2026-01-19 02:10:27.052229,apps_score_cash,v1.1,Quick,F,not applicable,android,Cash,apps_score_cash,Trench 1,Test,"{""app_cnt_rated_for_18plus_ever"": 0, ""app_cnt_...",0.511417,2026-01-19 10:10:22,2026-01-19 13:18:04,2026-01
506078,2962468,f1da90cc-230d-48c5-8623-54fcb171d620,2025-12-23T22:27:48.017316,2025-12-23T22:27:48.017316,apps_score_cash,v1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 2,Train,"{""beta_cash_app_score"": 0.49131212627705745, ""...",0.491312,2025-04-04 13:43:20,NaT,2025-04
143052,3516008,466b836b-6239-4d0c-aec7-e8c967707e05,2025-12-26 07:09:05.214863,2025-12-26 07:09:06.241505,apps_score_cash,v1.1,Quick,M,not applicable,android,Cash,apps_score_cash,Trench 2,Test,"{""app_cnt_rated_for_18plus_ever"": 0, ""app_cnt_...",0.483879,2025-12-26 15:09:00,2025-12-26 15:26:06,2025-12


### PSI calculation

In [111]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (78, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-10     score     Training   0.000000         406044             v1   
1   2025-09     score      Monthly   0.030436            521             v1   
2   2025-10     score      Monthly   0.009636          24667             v1   
3   2025-11     score      Monthly   0.021315          40688             v1   
4   2025-12     score      Monthly   0.027029          28521             v1   
5   2026-01     score      Monthly   4.993215           1965             v1   
6   2026-02     score      Monthly   4.864606            187             v1   
7   2025-10     score     Training   0.000000         271448             v1   
8   2025-09     score      Monthly   0.046215            378             v1   


In [112]:
print(f"The shape of the score_output_df is {score_output_df.shape}")
score_output_df['Model_Name'] = 'apps_score_cash'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'apps_score_cash_score')
score_output_df

The shape of the score_output_df is (78, 10)


,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-10,apps_score_cash_score,Training,0.000000,406044,v1,NA,NA,NA,NA,apps_score_cash
1,2025-09,apps_score_cash_score,Monthly,0.030436,521,v1,NA,NA,NA,NA,apps_score_cash
2,2025-10,apps_score_cash_score,Monthly,0.009636,24667,v1,NA,NA,NA,NA,apps_score_cash
3,2025-11,apps_score_cash_score,Monthly,0.021315,40688,v1,NA,NA,NA,NA,apps_score_cash
4,2025-12,apps_score_cash_score,Monthly,0.027029,28521,v1,NA,NA,NA,NA,apps_score_cash
5,2026-01,apps_score_cash_score,Monthly,4.993215,1965,v1,NA,NA,NA,NA,apps_score_cash
6,2026-02,apps_score_cash_score,Monthly,4.864606,187,v1,NA,NA,NA,NA,apps_score_cash
7,2025-10,apps_score_cash_score,Training,0.000000,271448,v1,Trench 1,NA,NA,NA,apps_score_cash
8,2025-09,apps_score_cash_score,Monthly,0.046215,378,v1,Trench 1,NA,NA,NA,apps_score_cash
9,2025-10,apps_score_cash_score,Monthly,0.016260,17704,v1,Trench 1,NA,NA,NA,apps_score_cash


In [113]:
psi_results = score_output_df.copy()
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_appscore_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result() 

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8729de4f-63d0-45b5-af5f-f3db96392218>

### Beta-Cash-Stack-Model

#### Test

In [114]:
sq = """
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in ('Beta-Cash-Stack-Model') then 'beta_stack_model_cash' else modelDisplayName end as modelDisplayName,
modelVersionId,
start_time,
end_time,
prediction,
case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
REPLACE(REPLACE(prediction, "'", '"'), "None", "null") AS prediction_clean
FROM `prj-prod-dataplatform.audit_balance.ml_model_run_details`
where modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction score
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'beta_stack_model_cash' Model_Name,
  trenchCategory,
   'Test' Data_selection,
  calcFeatures,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  r.start_time)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID 6b10485c-690a-4ec9-ae51-cff602ae64e7 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1002218,ed21f34d-0b01-4549-8038-50512c4d78d8,0.4330801845368297,2025-11-14 08:02:27.080302,2025-11-14 08:02:27.818778,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Test,"{""demo_score"": ""0.5901467966537366"", ""credo_sc...",2025-11-14 16:02:19,2025-11-15 20:01:29,2025-11
1,1014767,6ae06a14-c10c-4cf2-acf9-ffdd3a5c58e0,0.5469597067504398,2025-12-28 10:36:44.864980,2025-12-28 10:36:45.482898,beta_stack_model_cash,v1.1,Quick,F,not applicable,ios,Cash,beta_stack_model_cash,Trench 2,Test,"{""demo_score"": ""0.5907472430587256"", ""credo_sc...",2025-12-28 18:36:41,NaT,2025-12
2,1015021,c23d9e62-848f-4ab1-a26c-26ff4540cf76,0.32080781301888606,2025-11-17 12:17:50.712212,2025-11-17 12:17:51.239314,beta_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_stack_model_cash,Trench 2,Test,"{""demo_score"": ""0.4604157517155119"", ""credo_sc...",2025-11-17 20:17:47,NaT,2025-11
3,1043055,2fb9628e-858a-41f5-b6c2-319021e97881,0.1912668599317489,2025-12-12 14:56:56.286540,2025-12-12 14:56:57.072234,beta_stack_model_cash,v1.1,Quick,F,not applicable,ios,Cash,beta_stack_model_cash,Trench 2,Test,"{""demo_score"": ""0.4664731894861717"", ""credo_sc...",2025-12-12 22:56:53,NaT,2025-12
4,1048996,ac994292-0285-4388-b2c8-e0245dfe252d,0.19027721546640425,2025-11-14 14:12:06.771066,2025-11-14 14:12:07.630846,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Test,"{""demo_score"": ""0.41321850295420565"", ""credo_s...",2025-11-14 22:12:04,NaT,2025-11


In [115]:
df1 = dfd.copy()

#### Train

In [116]:
sq = """ 
WITH parsed as (
select customerId, digitalLoanAccountId,
case when modelDisplayName in ('Beta-Cash-Stack-Model') then 'beta_stack_model_cash' else modelDisplayName end modelDisplayName,
modelVersionId,start_time,end_time,prediction,
case when trenchCategory is null then 'ALL' 
         when trenchCategory='' then 'ALL'    
            else trenchCategory end trenchCategory,
REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeatures,
FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details
where modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
),
base as
(select 
 r.customerId,r.digitalLoanAccountId,prediction score
 ,start_time,end_time,
  modelDisplayName,modelVersionId,
     loanmaster.new_loan_type ,
 loanmaster.gender,
    case when loanmaster.loantype='BNPL' and sil_category.store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and sil_category.store_type =2 then 'Mobile'
    when loanmaster.loantype='BNPL' and sil_category.store_type =3 then 'Mall'
    when loanmaster.loantype='BNPL' and sil_category.store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
     case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
    when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
    when lower(loanmaster.deviceType) like '%andro%' then 'android'
    else 'ios' end osType,
 'Cash' as product,
 'beta_stack_model_cash' Model_Name,
  trenchCategory,
   'Train' Data_selection,
  calcFeatures,
  IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime) AS appln_submit_datetime,
  loanmaster.disbursementDateTime,
  format_date('%Y-%m',IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime)) as Application_month, 
from parsed r
left join risk_credit_mis.loan_master_table loanmaster
  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
 left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  qualify row_number() over(partition by r.customerId, r.digitalLoanAccountid order by cast(start_time as datetime) desc) = 1
)
select * from base where lower(new_loan_type) not like '%sil%'
;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
dfd.head()

Job ID 1e1fc6c3-adfa-4210-a75f-e252e32f88f1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
0,1003174,a1f11c4f-0595-43b6-89c6-2b9170d6a411,0.092268,2025-12-23T22:26:45.797325,2025-12-23T22:26:45.797325,beta_stack_model_cash,v1,Quick,M,not applicable,ios,Cash,beta_stack_model_cash,Trench 3,Train,"{""demo_score"": 0.30020523876987726, ""credo_sco...",2025-04-11 11:39:33,NaT,2025-04
1,1018638,4934e815-d590-444e-bd33-7973212d81c6,0.416922,2025-12-23T22:26:21.803676,2025-12-23T22:26:21.803676,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.5131903192879183, ""apps_score...",2025-04-03 14:13:44,2025-04-03 14:34:27,2025-04
2,1030436,d032c3ac-6b80-46ec-a83e-769cc9409074,0.313091,2025-12-23T22:26:25.303995,2025-12-23T22:26:25.303995,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.5720210774695103, ""apps_score...",2024-11-15 00:19:05,NaT,2024-11
3,1037620,0f33d94f-d82a-4c46-abb3-54d8e8956294,0.487352,2025-12-23T22:26:26.176998,2025-12-23T22:26:26.176998,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.632255731975995, ""apps_score""...",2024-11-12 05:49:37,NaT,2024-11
4,1038724,0cf737fd-d194-42ab-9b33-982413b8d18f,0.115376,2025-12-23T22:26:22.737975,2025-12-23T22:26:22.737975,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.4658799241977078, ""apps_score...",2025-09-04 22:03:33,2025-09-05 18:17:23,2025-09


In [117]:
df2 = dfd.copy()

In [118]:
df_concat = pd.concat([df1, df2], ignore_index=True)
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")

The shape of the concatenated dataframe is: (762401, 19)


In [119]:
print(f"The shape of the concatenated dataframe is: {df_concat.shape}")
df_combined = dropping_duplicates(df_concat)
print(f"The shape of the dataframe after dropping duplicates is: {df_combined.shape}")

The shape of the concatenated dataframe is: (762401, 19)
The shape of the dataframe after dropping duplicates is: (762401, 19)


In [120]:
df_combined['score'] = pd.to_numeric(df_combined['score'], errors='coerce')

In [121]:
df_combined.sample(10)

,customerId,digitalLoanAccountId,score,start_time,end_time,modelDisplayName,modelVersionId,new_loan_type,gender,loan_product_type,osType,product,Model_Name,trenchCategory,Data_selection,calcFeatures,appln_submit_datetime,disbursementDateTime,Application_month
641114,2526351,24ce73d6-35da-42e5-92b6-4da8e77360a1,0.712867,2025-12-23T22:26:42.935865,2025-12-23T22:26:42.935865,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 3,Train,"{""demo_score"": 0.4377308940732958, ""apps_score...",2024-12-18 09:17:59,NaT,2024-12
167791,3456363,60984fa0-c93e-49ba-b2ba-75cd6aec8818,0.390022,2025-10-08 09:42:18.641830,2025-10-08 09:42:19.586793,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Test,"{""demo_score"": ""0.5894536975316316"", ""credo_sc...",2025-10-08 17:42:15,NaT,2025-10
53839,3897407,db189f2a-0333-4f6d-b2ab-649d47f88ff3,0.572479,2025-12-14 12:40:51.759978,2025-12-14 12:40:52.441342,beta_stack_model_cash,v1.1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 1,Test,"{""demo_score"": ""0.5526564261367549"", ""credo_sc...",2025-12-14 20:40:48,2025-12-14 20:57:27,2025-12
3519,3485559,dac0fbda-5a03-4f5e-95fb-45e5c93b9283,0.122298,2025-11-23 00:51:20.144998,2025-11-23 00:51:20.876201,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Test,"{""demo_score"": ""0.44381187703642727"", ""credo_s...",2025-11-23 08:51:16,NaT,2025-11
474454,3662028,ec25abf9-e92d-4bdd-a021-d9eb41737930,0.208410,2025-12-23T22:25:45.771534,2025-12-23T22:25:45.771534,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.37568721634187713, ""apps_scor...",2025-09-03 17:32:06,NaT,2025-09
557712,2112325,9e65a4b6-c795-4d0d-8895-a411846f6416,0.764609,2025-12-23T22:26:26.518420,2025-12-23T22:26:26.518420,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 2,Train,"{""demo_score"": 0.5443622549868948, ""apps_score...",2025-03-29 13:28:02,NaT,2025-03
570581,3366120,9367a454-a876-4dca-b93f-8aa730eeebba,0.549101,2025-12-23T22:25:59.623482,2025-12-23T22:25:59.623482,beta_stack_model_cash,v1,Quick,F,not applicable,ios,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.530230107343834, ""credo_score...",2025-04-15 14:20:48,NaT,2025-04
620390,3595300,b369bfb9-e608-4b96-a2db-054dd5f6ed41,0.720438,2025-12-23T22:26:03.536610,2025-12-23T22:26:03.536610,beta_stack_model_cash,v1,Quick,M,not applicable,ios,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.6351461534902962, ""credo_scor...",2025-08-03 02:49:56,NaT,2025-08
687305,3323931,a18d724c-9c4e-4940-9060-395d06d3b8f7,0.619756,2025-12-23T22:25:53.600119,2025-12-23T22:25:53.600119,beta_stack_model_cash,v1,Quick,F,not applicable,android,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.6017273008177417, ""apps_score...",2025-03-16 17:55:09,NaT,2025-03
569125,3044678,1913a0d0-a47c-4eed-a5c5-16a07fa4d84e,0.434717,2025-12-23T22:25:55.237264,2025-12-23T22:25:55.237264,beta_stack_model_cash,v1,Quick,M,not applicable,android,Cash,beta_stack_model_cash,Trench 1,Train,"{""demo_score"": 0.5475414099348023, ""apps_score...",2024-11-22 02:16:12,NaT,2024-11


### PSI calculation

In [122]:
score_output_df = calculate_psi_with_segments(
    df=df_combined,
    score_column='score',
    date_column='Application_month',
    data_selection_column='Data_selection',
    segment_columns=['modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']
)

# This will create waterfall levels:
# Level 1: Only modelVersionId
# Level 2: modelVersionId + trenchCategory
# Level 3: modelVersionId + trenchCategory + new_loan_type
# Level 4: modelVersionId + trenchCategory + new_loan_type + loan_product_type
# Level 5: modelVersionId + trenchCategory + new_loan_type + loan_product_type + osType

print(f"Output shape: {score_output_df.shape}")
print(f"\nOutput columns: {score_output_df.columns.tolist()}")
print(f"\nFirst few rows:")
print(score_output_df.head(20))
print(f"\nData types:")
print(score_output_df.dtypes)

Output shape: (92, 10)

Output columns: ['Month', 'scorename', 'DateCategory', 'psivalues', 'account_count', 'modelVersionId', 'trenchCategory', 'new_loan_type', 'loan_product_type', 'osType']

First few rows:
      Month scorename DateCategory  psivalues  account_count modelVersionId  \
0   2025-10     score     Training   0.000000         488440             v1   
1   2025-09     score      Monthly   0.026940            803             v1   
2   2025-10     score      Monthly   0.006787          35272             v1   
3   2025-11     score      Monthly   0.009895          58553             v1   
4   2025-12     score      Monthly   0.009527          39042             v1   
5   2026-01     score      Monthly   2.376856           2898             v1   
6   2026-02     score      Monthly   2.470580            293             v1   
7   2025-10     score     Training   0.000000         324344             v1   
8   2025-09     score      Monthly   0.033305            578             v1   


In [123]:
print(f"The shape of the score_output_df is {score_output_df.shape}")
score_output_df['Model_Name'] = 'beta_stack_model_cash'
score_output_df['scorename'] = score_output_df['scorename'].replace('score', 'beta_stack_model_cash_score')
score_output_df

The shape of the score_output_df is (92, 10)


,Month,scorename,DateCategory,psivalues,account_count,modelVersionId,trenchCategory,new_loan_type,loan_product_type,osType,Model_Name
0,2025-10,beta_stack_model_cash_score,Training,0.000000,488440,v1,NA,NA,NA,NA,beta_stack_model_cash
1,2025-09,beta_stack_model_cash_score,Monthly,0.026940,803,v1,NA,NA,NA,NA,beta_stack_model_cash
2,2025-10,beta_stack_model_cash_score,Monthly,0.006787,35272,v1,NA,NA,NA,NA,beta_stack_model_cash
3,2025-11,beta_stack_model_cash_score,Monthly,0.009895,58553,v1,NA,NA,NA,NA,beta_stack_model_cash
4,2025-12,beta_stack_model_cash_score,Monthly,0.009527,39042,v1,NA,NA,NA,NA,beta_stack_model_cash
5,2026-01,beta_stack_model_cash_score,Monthly,2.376856,2898,v1,NA,NA,NA,NA,beta_stack_model_cash
6,2026-02,beta_stack_model_cash_score,Monthly,2.470580,293,v1,NA,NA,NA,NA,beta_stack_model_cash
7,2025-10,beta_stack_model_cash_score,Training,0.000000,324344,v1,Trench 1,NA,NA,NA,beta_stack_model_cash
8,2025-09,beta_stack_model_cash_score,Monthly,0.033305,578,v1,Trench 1,NA,NA,NA,beta_stack_model_cash
9,2025-10,beta_stack_model_cash_score,Monthly,0.004550,24909,v1,Trench 1,NA,NA,NA,beta_stack_model_cash


In [124]:
psi_results = score_output_df.copy()
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.beta_cash_stack_model_psi_v7"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(psi_results, table_id, job_config=job_config)
job.result() 

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e618dd40-291b-4178-802a-2c893b547d57>

# End